# 📊 Modelamiento Predictivo – Resultados ICFES Saber 11
### Aprendizaje de Máquinas | Universidad Pontificia Bolivariana

---

## Descripción del Proyecto

Pipeline completo de Machine Learning para predecir los puntajes del examen Saber 11
a partir de variables socioeconómicas, familiares e institucionales.

## Etapas del Pipeline

| # | Etapa | Descripción |
|---|-------|-------------|
| 1 | **Configuración** | Importaciones y constantes globales |
| 2 | **Carga de datos** | Lectura eficiente del CSV |
| 3 | **Exploración (EDA)** | Estadísticas, distribuciones y análisis de nulos |
| 4 | **Limpieza** | Normalización, corrección de valores y tipos de datos |
| 5 | **Imputación** | Tratamiento de valores faltantes |
| 6 | **Correlación** | Pearson (numéricas) y Cramér's V (categóricas) |
| 7 | **Feature Engineering** | Variables derivadas y eliminación de redundantes |
| 8 | **Codificación** | Ordinal y Label Encoding de categóricas |
| 9 | **Escalado** | MinMaxScaler [0, 1] para variables numéricas |
| 10 | **Dataset final** | Verificación y exportación para modelado |

## Variables

**Excluidas en carga:** identificadores ICFES, códigos DANE y campos administrativos

**Variables independientes (X):** características del estudiante, colegio y familia

**Variables dependientes (y):** `PUNT_GLOBAL`, `PUNT_MATEMATICAS`, `PUNT_INGLES`,
`PUNT_LECTURA_CRITICA`, `PUNT_C_NATURALES`, `PUNT_SOCIALES_CIUDADANAS`, `DESEMP_INGLES`


## ⚙️ 1. Configuración del Entorno

In [ ]:
# =============================================================================
# 1.1  IMPORTACIONES
# Todas las librerías se importan al inicio del notebook (buena práctica PEP8).
# =============================================================================

import unicodedata
import warnings
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder

# ── Configuración global ─────────────────────────────────────────────────────
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 60)
pd.set_option("display.float_format", "{:.4f}".format)
sns.set_theme(style="whitegrid", palette="muted", font_scale=1.1)
plt.rcParams.update({"figure.dpi": 120, "figure.figsize": (12, 5)})

print("✅ Librerías cargadas.")
print(f"   pandas {pd.__version__}  |  numpy {np.__version__}")


In [ ]:
# =============================================================================
# 1.2  CONSTANTES Y CONFIGURACIÓN CENTRALIZADA
# Centralizar rutas y listas de columnas evita "magic strings" dispersos y
# facilita el mantenimiento del código.
# =============================================================================

FILE_PATH = "/content/drive/MyDrive/AprendizajeMaquinas/Resultados_únicos_Saber_11_20260224.csv"

# ── Columnas administrativas a descartar en la carga ─────────────────────────
COLUMNS_TO_DROP: List[str] = [
    "ESTU_TIPODOCUMENTO", "ESTU_CONSECUTIVO",
    "COLE_COD_DANE_ESTABLECIMIENTO", "COLE_COD_DANE_SEDE",
    "COLE_COD_DEPTO_UBICACION", "COLE_COD_MCPIO_UBICACION",
    # COLE_DEPTO_UBICACION se conserva para filtrar por departamento en carga
    "COLE_CODIGO_ICFES", "COLE_GENERO", "COLE_NATURALEZA",
    "COLE_NOMBRE_ESTABLECIMIENTO", "COLE_NOMBRE_SEDE", "COLE_SEDE_PRINCIPAL",
    "ESTU_COD_DEPTO_PRESENTACION", "ESTU_COD_MCPIO_PRESENTACION",
    "ESTU_COD_RESIDE_DEPTO", "ESTU_COD_RESIDE_MCPIO",
    "ESTU_DEPTO_PRESENTACION", "ESTU_DEPTO_RESIDE",
    "ESTU_ESTADOINVESTIGACION", "ESTU_ESTUDIANTE",
    "ESTU_MCPIO_PRESENTACION", "ESTU_MCPIO_RESIDE",
    "ESTU_NACIONALIDAD", "ESTU_PAIS_RESIDE",
]

# ── Variables dependientes (targets a predecir) ───────────────────────────────
TARGET_COLS: List[str] = [
    "DESEMP_INGLES",
    "PUNT_INGLES",
    "PUNT_MATEMATICAS",
    "PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES",
    "PUNT_LECTURA_CRITICA",
    "PUNT_GLOBAL",
]

# Puntajes numéricos (subconjunto de targets, excluye la variable ordinal)
SCORE_COLS: List[str] = [t for t in TARGET_COLS if t != "DESEMP_INGLES"]

# Columnas críticas: se elimina la fila si alguna es nula
KEY_COLS: List[str] = ["PUNT_SOCIALES_CIUDADANAS", "PUNT_C_NATURALES", "PUNT_GLOBAL"]

# Representaciones textuales de "dato faltante"
NA_VALUES: List[str] = [
    "", " ", "NA", "N/A", "NULL", "null",
    "SIN DATO", "Sin dato", "-", "--", "No aplica", "NO APLICA",
]

# Variables binarias Sí/No del hogar
ASSET_COLS: List[str] = [
    "FAMI_TIENEAUTOMOVIL", "FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET",  "FAMI_TIENELAVADORA",
]
YN_COLS: List[str] = ASSET_COLS + ["COLE_BILINGUE"]

# Variables independientes finales (post feature engineering)
INDEPENDENT_VARS: List[str] = [
    "COLE_AREA_UBICACION", "COLE_BILINGUE", "COLE_CALENDARIO",
    "COLE_CARACTER", "COLE_JORNADA", "COLE_MCPIO_UBICACION",
    "ESTU_GENERO", "EDAD",
    "FAMI_CUARTOSHOGAR", "FAMI_EDUCACIONMADRE", "FAMI_EDUCACIONPADRE",
    "FAMI_ESTRATOVIVIENDA", "FAMI_PERSONASHOGAR", "INDICE_BIENES",
    "TRIMESTRE",
]

print("✅ Constantes del proyecto configuradas.")


## 📥 2. Carga del Dataset

In [ ]:
# =============================================================================
# 2.1  CARGA EFICIENTE DEL CSV — Solo registros del departamento QUINDIO
# =============================================================================

# Paso 1: Header
all_columns = pd.read_csv(FILE_PATH, nrows=0, encoding="latin-1").columns.tolist()

# Paso 2: Filtrado de columnas (conservar COLE_DEPTO_UBICACION para filtrar)
columns_to_use = [c for c in all_columns if c not in COLUMNS_TO_DROP]
print(f"Columnas en el archivo  : {len(all_columns)}")
print(f"Columnas descartadas    : {len(COLUMNS_TO_DROP)}")
print(f"Columnas que se cargan  : {len(columns_to_use)}")

# Paso 3: Carga completa
df_raw_full = pd.read_csv(
    FILE_PATH,
    usecols=columns_to_use,
    encoding="latin-1",
    low_memory=False,
)

# Paso 4: Filtrar solo QUINDIO
# Normalizar el campo para comparacion robusta (sin tildes, mayusculas)
import unicodedata
def strip_accents(s):
    if pd.isna(s): return s
    return ''.join(c for c in unicodedata.normalize('NFD', str(s))
                   if unicodedata.category(c) != 'Mn').upper().strip()

if "COLE_DEPTO_UBICACION" in df_raw_full.columns:
    depto_norm = df_raw_full["COLE_DEPTO_UBICACION"].apply(strip_accents)
    mask_quindio = depto_norm == "QUINDIO"
    df_raw = df_raw_full[mask_quindio].copy().reset_index(drop=True)
    print(f"\n  Registros totales en CSV : {len(df_raw_full):>10,}")
    print(f"  Registros QUINDIO        : {len(df_raw):>10,}  ({len(df_raw)/len(df_raw_full)*100:.2f}%)")
    # Eliminar la columna de departamento (ya no es util como variable)
    df_raw.drop(columns=["COLE_DEPTO_UBICACION"], inplace=True, errors="ignore")
else:
    df_raw = df_raw_full.copy()
    print("  AVISO: COLE_DEPTO_UBICACION no encontrada, se carga dataset completo")

mem_mb = df_raw.memory_usage(deep=True).sum() / 1024**2
print(f"\n Dataset final cargado:")
print(f"   Filas     : {df_raw.shape[0]:>12,}")
print(f"   Columnas  : {df_raw.shape[1]:>12,}")
print(f"   Memoria   : {mem_mb:>11.1f} MB")


## 🔍 3. Exploración de Datos (EDA)

El Análisis Exploratorio de Datos permite entender la estructura del dataset,
detectar problemas de calidad y guiar las decisiones de preprocesamiento.


In [ ]:
# =============================================================================
# 3.1  INFORMACIÓN GENERAL
# =============================================================================
print("=" * 65)
print(" TIPOS DE DATOS Y VALORES NO NULOS")
print("=" * 65)
df_raw.info(verbose=True, show_counts=True)


In [ ]:
# =============================================================================
# 3.2  ESTADÍSTICAS DESCRIPTIVAS
# =============================================================================
print("── Variables numéricas ────────────────────────────────────────")
display(df_raw.describe().round(2))

print("\n── Variables categóricas (object) ─────────────────────────────")
display(df_raw.describe(include=["object"]))


In [ ]:
# =============================================================================
# 3.3  ANÁLISIS DE VALORES NULOS
# =============================================================================
null_df = pd.DataFrame({
    "Nulos"   : df_raw.isnull().sum(),
    "% Nulos" : (df_raw.isnull().mean() * 100).round(2),
    "Tipo"    : df_raw.dtypes,
    "Únicos"  : df_raw.nunique(),
}).sort_values("% Nulos", ascending=False)

print("── Columnas con valores nulos ──────────────────────────────────")
display(null_df[null_df["Nulos"] > 0])

# Heatmap de nulos (muestra de 5 000 filas)
sample_nulls = df_raw.sample(min(5000, len(df_raw)), random_state=42).isnull()
cols_w_nulls = sample_nulls.columns[sample_nulls.any()].tolist()

if cols_w_nulls:
    fig, ax = plt.subplots(figsize=(16, 5))
    sns.heatmap(sample_nulls[cols_w_nulls].T, cbar=False, cmap="YlOrRd",
                ax=ax, xticklabels=False)
    ax.set_title("Mapa de calor de valores nulos (muestra 5 000 filas)", fontsize=13)
    ax.set_xlabel("Observaciones"); ax.set_ylabel("Variables")
    plt.tight_layout(); plt.savefig("heatmap_nulos.png", dpi=130, bbox_inches="tight")
    plt.show()


In [ ]:
# =============================================================================
# 3.4  DISTRIBUCIÓN DE VARIABLES TARGET (PUNTAJES)
# =============================================================================
score_present = [c for c in SCORE_COLS if c in df_raw.columns]
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for i, col in enumerate(score_present):
    data = df_raw[col].dropna()
    axes[i].hist(data, bins=50, color="#4C72B0", edgecolor="white", linewidth=0.5)
    axes[i].axvline(data.mean(),   color="#DD8452", lw=2, ls="--", label=f"Media {data.mean():.1f}")
    axes[i].axvline(data.median(), color="#55A868", lw=2, ls=":",  label=f"Mediana {data.median():.1f}")
    axes[i].set_title(col, fontsize=11, fontweight="bold")
    axes[i].set_xlabel("Puntaje"); axes[i].set_ylabel("Frecuencia")
    axes[i].legend(fontsize=8)

for j in range(len(score_present), len(axes)):
    axes[j].set_visible(False)

fig.suptitle("Distribución de Variables Target (Puntajes)", fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig("distribucion_targets.png", dpi=130, bbox_inches="tight")
plt.show()


In [ ]:
# =============================================================================
# 3.5  DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS CLAVE
# =============================================================================
cat_eda = [c for c in [
    "ESTU_GENERO", "COLE_AREA_UBICACION", "COLE_BILINGUE",
    "COLE_CALENDARIO", "COLE_JORNADA", "FAMI_ESTRATOVIVIENDA",
    "FAMI_EDUCACIONMADRE", "FAMI_EDUCACIONPADRE",
] if c in df_raw.columns]

fig, axes = plt.subplots(2, 4, figsize=(18, 9))
axes = axes.flatten()
palette = sns.color_palette("muted")

for i, col in enumerate(cat_eda):
    vc = df_raw[col].value_counts(dropna=False).head(10)
    vc.plot(kind="bar", ax=axes[i], color=palette, edgecolor="white")
    axes[i].set_title(col, fontsize=10, fontweight="bold")
    axes[i].tick_params(axis="x", rotation=35, labelsize=8)
    axes[i].set_ylabel("Frecuencia" if i % 4 == 0 else "")

for j in range(len(cat_eda), len(axes)):
    axes[j].set_visible(False)

fig.suptitle("Distribución de Variables Categóricas Clave", fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig("distribucion_categoricas.png", dpi=130, bbox_inches="tight")
plt.show()


## 🧹 4. Limpieza de Datos

Se corrigen inconsistencias de formato, se normalizan valores y se ajustan
los tipos de datos de cada variable a los correctos para el modelado.


In [ ]:
# =============================================================================
# 4.1  ELIMINACIÓN DE FILAS SIN PUNTAJES CLAVE
# Los registros sin puntaje objetivo no pueden ser imputados de forma
# confiable ya que son exactamente lo que queremos predecir.
# =============================================================================
df = df_raw.copy()
initial_rows = len(df)
df = df.dropna(subset=KEY_COLS).copy()
removed = initial_rows - len(df)

print(f"Filas originales  : {initial_rows:>10,}")
print(f"Filas conservadas : {len(df):>10,}  ({len(df)/initial_rows*100:.2f}%)")
print(f"Filas eliminadas  : {removed:>10,}  ({removed/initial_rows*100:.2f}%)")


In [ ]:
# =============================================================================
# 4.2  NORMALIZACIÓN DE CADENAS NULAS Y ESPACIOS
# =============================================================================

# Reemplazar representaciones textuales de nulo por NaN estándar de pandas
df = df.replace(NA_VALUES, np.nan)

# Eliminar espacios al inicio/final en columnas de texto
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype("string").str.strip()

print("✅ Cadenas nulas normalizadas y espacios limpiados.")


In [ ]:
# =============================================================================
# 4.3  FUNCIONES AUXILIARES DE NORMALIZACIÓN
# =============================================================================

def remove_accents(text) -> str:
    """Elimina tildes y diacríticos de un string."""
    if pd.isna(text):
        return text
    return "".join(
        c for c in unicodedata.normalize("NFKD", str(text))
        if not unicodedata.combining(c)
    )

def norm_gender(series: pd.Series) -> pd.Series:
    """Normaliza género a 'M' / 'F'. Valores desconocidos → NaN."""
    x = series.astype("string").str.upper().str.strip()
    return x.replace({
        "MASCULINO": "M", "FEMENINO": "F",
        "HOMBRE":    "M", "MUJER":    "F",
        "MALE":      "M", "FEMALE":   "F",
    }).where(lambda s: s.isin(["M", "F"]), np.nan)

def norm_yesno(series: pd.Series) -> pd.Series:
    """Normaliza variable binaria a 'SI' / 'NO'. Valores desconocidos → NaN."""
    x = series.astype("string").str.upper().str.strip()
    return x.replace({
        "SÍ": "SI", "YES": "SI", "Y": "SI", "1": "SI", "TRUE": "SI",
        "N":  "NO", "0":   "NO",             "FALSE": "NO",
    }).where(lambda s: s.isin(["SI", "NO"]), np.nan)

def fix_encoding(text):
    """Intenta corregir texto latin-1 decodificado incorrectamente como utf-8."""
    if pd.isna(text):
        return text
    try:
        return str(text).encode("latin1").decode("utf-8")
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text

print("✅ Funciones auxiliares definidas.")


In [ ]:
# =============================================================================
# 4.4  APLICACION DE NORMALIZACIONES
# =============================================================================

# Correccion de encoding en texto
for col in df.select_dtypes(include=["object", "string"]).columns:
    df[col] = df[col].apply(fix_encoding)

# Genero
df["ESTU_GENERO"] = norm_gender(df["ESTU_GENERO"])

# Variables Si/No
for col in YN_COLS:
    if col in df.columns:
        df[col] = norm_yesno(df[col])

# COLE_BILINGUE: vacio, NaN o cadena vacia = SI (regla de negocio)
df["COLE_BILINGUE"] = (
    df["COLE_BILINGUE"]
    .astype("string")
    .str.strip()
    .replace({"": "SI", "<NA>": "SI", "None": "SI", "nan": "SI"})
)
df["COLE_BILINGUE"] = df["COLE_BILINGUE"].fillna("SI")
# Doble verificacion: cualquier valor que no sea NO se considera SI
df["COLE_BILINGUE"] = df["COLE_BILINGUE"].apply(
    lambda x: "NO" if str(x).strip().upper() == "NO" else "SI"
)

# COLE_CARACTER: unificar variantes tipograficas (sin tildes)
df["COLE_CARACTER"] = (
    df["COLE_CARACTER"].astype("string").str.upper()
    .str.replace("TECNICO/ACADEMICO", "TECNICO-ACADEMICO", regex=False)
    .str.replace("TECNICO", "TECNICO", regex=False)
    .apply(remove_accents)
)

# COLE_JORNADA: quitar tildes y caracteres especiales
if "COLE_JORNADA" in df.columns:
    df["COLE_JORNADA"] = (
        df["COLE_JORNADA"].astype("string").str.upper().str.strip()
        .apply(remove_accents)
    )

# FAMI_EDUCACIONMADRE y FAMI_EDUCACIONPADRE: quitar tildes
for col in ["FAMI_EDUCACIONMADRE", "FAMI_EDUCACIONPADRE"]:
    if col in df.columns:
        df[col] = (
            df[col].astype("string").str.strip()
            .apply(remove_accents)
        )

# Departamento: mayusculas y sin tildes para consistencia
if "COLE_DEPTO_UBICACION" in df.columns:
    df["COLE_DEPTO_UBICACION"] = (
        df["COLE_DEPTO_UBICACION"].astype("string").str.upper().apply(remove_accents)
    )

print("Normalizaciones aplicadas.")
print("  ESTU_GENERO   :", df["ESTU_GENERO"].value_counts(dropna=False).to_dict())
print("  COLE_BILINGUE :", df["COLE_BILINGUE"].value_counts(dropna=False).to_dict())
print("  COLE_JORNADA  :", df["COLE_JORNADA"].value_counts(dropna=False).to_dict() if "COLE_JORNADA" in df.columns else "no disponible")
print("  EDU_MADRE vals:", df["FAMI_EDUCACIONMADRE"].dropna().unique()[:5].tolist() if "FAMI_EDUCACIONMADRE" in df.columns else "no disponible")


In [ ]:
# =============================================================================
# 4.5  CONVERSIÓN DE TIPOS DE DATOS
# Se asigna el dtype correcto a cada variable para garantizar cómputos válidos
# y optimizar el uso de memoria.
# =============================================================================

# ── Puntajes numéricos → float64 ──────────────────────────────────────────────
for col in SCORE_COLS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# ── Personas en el hogar → numérico (puede venir como rango "3 a 4") ─────────
if "FAMI_PERSONASHOGAR" in df.columns:
    df["FAMI_PERSONASHOGAR"] = (
        df["FAMI_PERSONASHOGAR"].astype("string").str.upper()
        .str.extract(r"(\d+)", expand=False)
    )
    df["FAMI_PERSONASHOGAR"] = pd.to_numeric(df["FAMI_PERSONASHOGAR"], errors="coerce")

# ── Fecha de nacimiento → datetime ───────────────────────────────────────────
df["ESTU_FECHANACIMIENTO"] = pd.to_datetime(
    df["ESTU_FECHANACIMIENTO"], errors="coerce", dayfirst=True
)

# ── Variables categóricas → dtype category (ahorra memoria en baja cardinalidad) ──
for col in df.select_dtypes(include=["object", "string"]).columns:
    if df[col].nunique(dropna=False) < 200:
        df[col] = df[col].astype("category")

print("✅ Tipos de datos convertidos:")
print(df.dtypes.value_counts().to_string())
print(f"\n📉 Memoria tras conversión: {df.memory_usage(deep=True).sum()/1024**2:.1f} MB")


### 🗑️ 4.6 — Eliminar columnas con más del 50% de valores nulos

Columnas donde la mitad o más de los registros son nulos no pueden imputarse
de forma confiable y añaden ruido al modelo. Se eliminan antes de la imputación.

In [ ]:
# =============================================================================
# 4.6  ELIMINAR COLUMNAS CON > 50% DE NULOS
#
# Umbral: si una columna tiene más del 50% de sus valores como NaN,
# se descarta. Imputar más del 50% de los datos es estadísticamente
# indefendible y puede introducir más sesgo que información.
# Se excluyen siempre las columnas TARGET_COLS para no perder variables obj.
# =============================================================================

NULL_THRESHOLD = 0.50

n_rows = len(df)
null_ratios = df.drop(columns=[c for c in TARGET_COLS if c in df.columns],
                      errors='ignore').isnull().mean()

cols_high_null = null_ratios[null_ratios > NULL_THRESHOLD].index.tolist()

print(f'Umbral de nulos: > {NULL_THRESHOLD*100:.0f}%')
print(f'Columnas analizadas: {len(null_ratios)}')
print(f'Columnas a eliminar: {len(cols_high_null)}')
print()

if cols_high_null:
    detalle = pd.DataFrame({
        'Columna'  : cols_high_null,
        '% Nulos'  : [round(null_ratios[c]*100, 2) for c in cols_high_null],
        'Nulos'    : [int(null_ratios[c]*n_rows) for c in cols_high_null],
        'No nulos' : [int((1-null_ratios[c])*n_rows) for c in cols_high_null],
    }).sort_values('% Nulos', ascending=False).reset_index(drop=True)
    display(detalle)

    df = df.drop(columns=cols_high_null)

    # Actualizar INDEPENDENT_VARS si alguna fue eliminada
    INDEPENDENT_VARS = [v for v in INDEPENDENT_VARS if v not in cols_high_null]

    print(f'\nDataFrame tras eliminar columnas muy nulas: {df.shape}')
    print(f'INDEPENDENT_VARS actualizado: {len(INDEPENDENT_VARS)} variables')
else:
    print('No hay columnas con mas del 50% de nulos. No se elimino nada.')

print(f'\nColumnas conservadas: {df.shape[1]}')


### 🗑️ 4.7 — Eliminar columnas quasi-constantes (> 90% un solo valor)

Variables donde el 90%+ de los registros tienen el mismo valor aportan
información mínima al modelo y pueden causar problemas de varianza cero
en validación cruzada. Se eliminan antes de la imputación.

> **Umbral:** > 90% (más estricto que el 95% del diagnóstico R10 en §5,
> que era solo informativo). Aquí sí se aplica la eliminación.

In [ ]:
# =============================================================================
# 4.7  ELIMINAR COLUMNAS QUASI-CONSTANTES (> 90% UN SOLO VALOR)
#
# Una variable donde el 90%+ de los registros tienen el mismo valor
# no aporta poder discriminativo al modelo y puede causar:
#   - Varianza cero en folds de CV  -> error en algunos estimadores
#   - Coeficientes inflados en Ridge/ElasticNet
#   - Splits inutiles en arboles
# Se excluyen siempre TARGET_COLS.
# =============================================================================

QC_THRESHOLD = 0.90

excluir_qc = set(TARGET_COLS) | {'ESTU_FECHANACIMIENTO', 'PERIODO'}
cols_eval  = [c for c in df.columns if c not in excluir_qc]

qc_resultados = []
cols_qc = []

for col in cols_eval:
    try:
        vc       = df[col].value_counts(normalize=True, dropna=False)
        if len(vc) == 0:
            continue
        top_ratio = float(vc.iloc[0])
        top_val   = vc.index[0]
        qc_resultados.append({
            'Columna'       : col,
            'Valor dominante': str(top_val)[:40],
            '% dominante'   : round(top_ratio * 100, 2),
            'Valores unicos': int(df[col].nunique()),
            'Eliminar'      : top_ratio > QC_THRESHOLD,
        })
        if top_ratio > QC_THRESHOLD:
            cols_qc.append(col)
    except Exception:
        continue

df_qc = (pd.DataFrame(qc_resultados)
         .sort_values('% dominante', ascending=False)
         .reset_index(drop=True))

print(f'Umbral quasi-constante: > {QC_THRESHOLD*100:.0f}%')
print(f'Columnas evaluadas: {len(df_qc)}')
print(f'Columnas a eliminar: {len(cols_qc)}')
print()

if cols_qc:
    display(df_qc[df_qc['Eliminar'] == True]
            [['Columna','Valor dominante','% dominante','Valores unicos']])
    df = df.drop(columns=cols_qc)
    INDEPENDENT_VARS = [v for v in INDEPENDENT_VARS if v not in cols_qc]
    print(f'\nDataFrame tras eliminar quasi-constantes: {df.shape}')
    print(f'INDEPENDENT_VARS actualizado: {len(INDEPENDENT_VARS)} variables')
else:
    print('No hay columnas quasi-constantes (>90%). No se elimino nada.')

# Visualizacion top 20 por % dominante
top_vis = df_qc.head(20)
fig, ax = plt.subplots(figsize=(10, max(4, len(top_vis)*0.38)))
colors_vis = ['#C44E52' if e else '#4C72B0' for e in top_vis['Eliminar']]
ax.barh(top_vis['Columna'], top_vis['% dominante'],
        color=colors_vis, edgecolor='white', height=0.65)
ax.axvline(QC_THRESHOLD*100, color='orange', lw=1.8, ls='--',
           label=f'Umbral {QC_THRESHOLD*100:.0f}%')
ax.set_xlabel('% valor dominante')
ax.set_title('4.7 Quasi-constantes (rojo = eliminadas, azul = conservadas)')
ax.legend(fontsize=9)
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('cleaning_quasi_constantes.png', dpi=120, bbox_inches='tight')
plt.show()
print('cleaning_quasi_constantes.png guardado')


---
## 🛡️ 5. Validación de Calidad de Datos

Las reglas se aplican sobre `df` **post-limpieza (§4) y pre-imputación (§6)**.
En este punto los datos están en sus **unidades originales** — sin EDAD calculada,
sin INDICE_BIENES, sin encoding ni escalado.

| Regla | Qué valida | Variable(s) | Acción si falla |
|-------|-----------|-------------|-----------------|
| R01 | Dominio `COLE_AREA_UBICACION` | `df` original | → NaN (imputar luego) |
| R02 | Consistencia geográfica municipio–departamento | `df` original | Solo reporte |
| R03 | Dominio `COLE_CALENDARIO` | `df` original | → NaN (imputar luego) |
| R04 | Dominio `COLE_CARACTER` | `df` original | → NaN (imputar luego) |
| R05 | Rango puntajes targets [0, 500] | `PUNT_*` originales | → NaN |
| R06 | Coherencia catálogo educación padres | Categorías originales | Solo reporte |
| R07 | Dominio `FAMI_ESTRATOVIVIENDA` | Valores originales | → NaN (imputar luego) |
| R08 | Variables numéricas con valores negativos | Numéricas originales | → NaN |
| R09 | Multicolinealidad categórica (Cramér's V > 0.80) | Categóricas originales | Reporte y recomendación |
| R10 | Variables casi constantes (> 95%) | Todas las X disponibles | Reporte |

> **Variables que NO existen aún en este punto:**
> `EDAD` (se crea en §6), `ANIO`, `TRIMESTRE` (§6), `INDICE_BIENES` (§7).
> Las reglas que las necesitarían se omiten o adaptan para usar alternativas disponibles.


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 5.0  MOTOR DE REPORTE DE VALIDACIONES
#      Se trabaja exclusivamente sobre `df` (post-limpieza, pre-imputación).
#      Las correcciones → NaN se propagan automáticamente a la imputación.
# ─────────────────────────────────────────────────────────────────────────────
from dataclasses import dataclass
from typing import List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

@dataclass
class VRule:
    rule_id  : str
    name     : str
    passed   : bool
    affected : int
    total    : int
    action   : str
    detail   : str = ""

    @property
    def pct(self):
        return round(self.affected / self.total * 100, 3) if self.total else 0.0

    def __str__(self):
        st = "✅ PASS" if self.passed else "⚠️  FAIL"
        return (
            f"{st}  [{self.rule_id}] {self.name}\n"
            f"        Afectados : {self.affected:,} / {self.total:,}  ({self.pct} %)\n"
            f"        Acción    : {self.action}"
            + (f"\n        Detalle   : {self.detail}" if self.detail else "")
        )

class DQReport:
    def __init__(self): self.rules: List[VRule] = []

    def add(self, r: VRule):
        self.rules.append(r)
        print(r)

    def summary(self) -> pd.DataFrame:
        return pd.DataFrame([{
            "ID"        : r.rule_id,
            "Regla"     : r.name,
            "Estado"    : "✅ PASS" if r.passed else "⚠️  FAIL",
            "Afectados" : r.affected,
            "% Afect."  : r.pct,
            "Acción"    : r.action,
        } for r in self.rules])

    def print_summary(self):
        ok = sum(1 for r in self.rules if r.passed)
        print(f"\n{'='*60}")
        print(f"  DQ REPORT  |  {ok}/{len(self.rules)} reglas aprobadas")
        print(f"  DataFrame : df  (post-§4 limpieza, pre-§6 imputación)")
        print(f"{'='*60}")

dq = DQReport()
print(f"✅ Motor DQReport listo.")
print(f"   df.shape en este punto: {df.shape}")
print(f"   Columnas: {list(df.columns)}")


### 🔹 R01 — Dominio válido: `COLE_AREA_UBICACION`

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R01  DOMINIO VÁLIDO — COLE_AREA_UBICACION
#      Dominio oficial MEN: {URBANO, RURAL}
#      Valores fuera → NaN para que §6 impute con la moda.
# ─────────────────────────────────────────────────────────────────────────────
VALID_AREA = {"URBANO", "RURAL"}
COL = "COLE_AREA_UBICACION"

if COL in df.columns:
    vals = df[COL].astype(str).str.strip().str.upper()
    mask = ~vals.isin(VALID_AREA) & ~vals.isin({"NAN", "NONE", "<NA>"})
    n    = int(mask.sum())

    print(f"Distribución de {COL} (valores originales):")
    vc = vals.value_counts(dropna=False).reset_index()
    vc.columns = ["Valor", "Conteo"]
    vc["En dominio"] = vc["Valor"].isin(VALID_AREA).map({True: "✅", False: "❌"})
    display(vc)

    if n:
        df.loc[mask, COL] = np.nan
        print(f"\n⚠️  {n:,} valores fuera del dominio corregidos → NaN")
    else:
        print(f"\n✅ Todos los valores están en el dominio.")

    dq.add(VRule("R01", f"Dominio {COL}", n == 0, n, len(df),
                 "→ NaN, imputar en §6" if n else "Sin acción",
                 f"Dominio: {VALID_AREA}"))
else:
    print(f"⚠️  {COL} no presente en df.")
    dq.add(VRule("R01", f"Dominio {COL}", True, 0, 0, "Columna no presente"))


### 🔹 R02 — Consistencia geográfica municipio–departamento

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R02  CONSISTENCIA GEOGRÁFICA — MUNICIPIO ↔ DEPARTAMENTO
#      Cada municipio debe pertenecer a un único departamento.
#      Múltiples departamentos para el mismo municipio = error en la fuente.
#      No hay corrección automática: requiere revisión de la fuente.
# ─────────────────────────────────────────────────────────────────────────────
COL_M = "COLE_MCPIO_UBICACION"
COL_D = "COLE_DEPTO_UBICACION"

if COL_M in df.columns and COL_D in df.columns:
    mdc = (
        df.dropna(subset=[COL_M, COL_D])
        .groupby(COL_M, observed=True)[COL_D]
        .nunique()
        .rename("n_deptos")
        .reset_index()
    )
    bad = mdc[mdc["n_deptos"] > 1]
    n   = len(bad)

    print(f"Municipios únicos    : {df[COL_M].nunique():,}")
    print(f"Departamentos únicos : {df[COL_D].nunique():,}")
    print(f"Municipios evaluados : {len(mdc):,}")

    if n:
        print(f"\n⚠️  {n} municipio(s) asociados a >1 departamento:")
        display(bad.head(15))
    else:
        print("\n✅ Cada municipio pertenece a un único departamento.")

    dq.add(VRule("R02", "Consistencia geográfica municipio–departamento",
                 n == 0, n, len(mdc),
                 "Revisar fuente de datos" if n else "Sin acción",
                 f"{df[COL_M].nunique()} municipios / {df[COL_D].nunique()} departamentos"))
else:
    print("⚠️  Columnas geográficas no disponibles.")
    dq.add(VRule("R02", "Consistencia geográfica", True, 0, 0, "No evaluable"))


### 🔹 R03 — Dominio válido: `COLE_CALENDARIO`

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R03  DOMINIO VÁLIDO — COLE_CALENDARIO
#      Valores oficiales MEN: A, B, OTRO
#      Variantes comunes normalizadas primero; resto → NaN.
# ─────────────────────────────────────────────────────────────────────────────
VALID_CAL = {"A", "B", "OTRO"}
COL = "COLE_CALENDARIO"

if COL in df.columns:
    df[COL] = df[COL].astype(str).str.strip().str.upper()
    # Normalizar variantes comunes antes de evaluar
    df[COL].replace({
        "CALENDARIO A": "A", "CALENDARIO_A": "A",
        "CALENDARIO B": "B", "CALENDARIO_B": "B",
    }, inplace=True)

    vals = df[COL]
    mask = ~vals.isin(VALID_CAL) & ~vals.isin({"NAN", "NONE", "<NA>"})
    n    = int(mask.sum())

    vc = vals.value_counts(dropna=False).reset_index()
    vc.columns = ["Valor", "Conteo"]
    vc["En dominio"] = vc["Valor"].isin(VALID_CAL).map({True: "✅", False: "❌"})
    display(vc)

    if n:
        df.loc[mask, COL] = np.nan
        print(f"\n⚠️  {n:,} registros irregulares → NaN")
    else:
        print("\n✅ Todos los valores en el dominio.")

    dq.add(VRule("R03", f"Dominio {COL}", n == 0, n, len(df),
                 "→ NaN, imputar en §6" if n else "Sin acción",
                 f"Dominio: {VALID_CAL}"))
else:
    dq.add(VRule("R03", f"Dominio {COL}", True, 0, 0, "No disponible"))


### 🔹 R04 — Dominio válido: `COLE_CARACTER`

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R04  DOMINIO VÁLIDO — COLE_CARACTER
#      Valores oficiales: ACADEMICO, TECNICO, TECNICO-ACADEMICO
# ─────────────────────────────────────────────────────────────────────────────
VALID_CAR = {"ACADEMICO", "TECNICO", "TECNICO-ACADEMICO"}
COL = "COLE_CARACTER"

if COL in df.columns:
    df[COL] = df[COL].astype(str).str.strip().str.upper()
    mask = ~df[COL].isin(VALID_CAR) & ~df[COL].isin({"NAN", "NONE", "<NA>"})
    n    = int(mask.sum())

    vc = df[COL].value_counts(dropna=False).reset_index()
    vc.columns = ["Valor", "Conteo"]
    vc["En dominio"] = vc["Valor"].isin(VALID_CAR).map({True: "✅", False: "❌"})
    display(vc)

    if n:
        df.loc[mask, COL] = np.nan
        print(f"\n⚠️  {n:,} registros fuera del dominio → NaN")
    else:
        print("\n✅ Todos los valores en el dominio.")

    dq.add(VRule("R04", f"Dominio {COL}", n == 0, n, len(df),
                 "→ NaN, imputar en §6" if n else "Sin acción",
                 f"Dominio: {VALID_CAR}"))
else:
    dq.add(VRule("R04", f"Dominio {COL}", True, 0, 0, "No disponible"))


### 🔹 R05 — Puntajes targets: valores fuera del rango esperado

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R05  RANGO VÁLIDO — PUNTAJES NUMÉRICOS (PUNT_*)
#
#  NOTA: EDAD no existe aún (se calcula en §6 Imputación).
#        Esta regla valida los PUNTAJES que sí existen en df post-§4.
#
#  Rangos esperados (ICFES oficial):
#    PUNT_GLOBAL              : 0 – 500
#    PUNT_* (por área)        : 0 – 100
#
#  Acción: valores imposibles → NaN (se excluyen del entrenamiento).
# ─────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt

RANGO = {
    "PUNT_GLOBAL"              : (0, 500),
    "PUNT_MATEMATICAS"         : (0, 100),
    "PUNT_INGLES"              : (0, 100),
    "PUNT_LECTURA_CRITICA"     : (0, 100),
    "PUNT_C_NATURALES"         : (0, 100),
    "PUNT_SOCIALES_CIUDADANAS" : (0, 100),
}

total_bad = 0
registros = {}

print(f"{'Variable':<35}  {'Min obs':>8}  {'Max obs':>8}  {'< mín':>8}  {'> máx':>8}  Estado")
print("─" * 85)

for col, (lo, hi) in RANGO.items():
    if col not in df.columns:
        continue
    s   = pd.to_numeric(df[col], errors="coerce")
    neg = int((s < lo).sum())
    ove = int((s > hi).sum())
    bad = neg + ove
    total_bad += bad

    flag = "✅" if bad == 0 else "⚠️ "
    print(f"{flag} {col:<33}  {s.min():>8.1f}  {s.max():>8.1f}  {neg:>8,}  {ove:>8,}  "
          + ("OK" if bad == 0 else f"{bad:,} → NaN"))

    if neg: df.loc[s < lo, col] = np.nan
    if ove: df.loc[s > hi, col] = np.nan

    registros[col] = {
        "min_obs": float(s.min()),
        "max_obs": float(s.max()),
        "n_bad"  : bad,
    }

# Visualización de distribuciones
cols_plot = [c for c in RANGO if c in df.columns]
fig, axes = plt.subplots(1, len(cols_plot), figsize=(4 * len(cols_plot), 4))
if len(cols_plot) == 1: axes = [axes]

for ax, col in zip(axes, cols_plot):
    lo, hi = RANGO[col]
    s = pd.to_numeric(df[col], errors="coerce").dropna()
    s.hist(bins=40, ax=ax, color="#4C72B0", edgecolor="white")
    ax.axvline(lo, color="#C44E52", lw=1.5, ls="--", label=f"min={lo}")
    ax.axvline(hi, color="#C44E52", lw=1.5, ls="--", label=f"max={hi}")
    ax.set_title(col.replace("PUNT_", ""), fontsize=9)
    ax.tick_params(labelsize=7)

fig.suptitle("R05 — Distribución de puntajes (valores originales, post-§4)", fontsize=11)
plt.tight_layout()
plt.savefig("r05_puntajes.png", dpi=120, bbox_inches="tight")
plt.show()

if total_bad == 0:
    print("\n✅ Todos los puntajes están dentro de los rangos esperados.")
else:
    print(f"\n⚠️  {total_bad:,} valores imposibles corregidos → NaN")

dq.add(VRule("R05", "Puntajes targets en rango esperado",
             total_bad == 0, total_bad, len(df),
             f"→ NaN: {total_bad:,} valores imposibles" if total_bad else "Sin acción",
             "PUNT_GLOBAL [0,500] | áreas [0,100]"))


### 🔹 R06 — Coherencia del catálogo de educación de padres

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R06  COHERENCIA CATÁLOGOS — EDUCACIÓN MADRE Y PADRE
#      Ambas columnas deben usar el mismo vocabulario de categorías.
#      Discrepancia → error de captura / fuente inconsistente.
#      Acción: solo reporte (no hay corrección automática posible).
# ─────────────────────────────────────────────────────────────────────────────
COL_M = "FAMI_EDUCACIONMADRE"
COL_P = "FAMI_EDUCACIONPADRE"

if COL_M in df.columns and COL_P in df.columns:
    EXCL = {"nan", "none", "<na>", ""}

    cats_m = {str(v).strip() for v in df[COL_M].dropna().unique()} - EXCL
    cats_p = {str(v).strip() for v in df[COL_P].dropna().unique()} - EXCL
    solo_m = cats_m - cats_p
    solo_p = cats_p - cats_m
    n_diff = len(solo_m) + len(solo_p)

    print(f"Categorías EDUCACIONMADRE : {len(cats_m)}")
    print(f"Categorías EDUCACIONPADRE : {len(cats_p)}")
    print(f"Categorías comunes        : {len(cats_m & cats_p)}")

    if solo_m: print(f"\n⚠️  Solo en MADRE ({len(solo_m)}): {sorted(solo_m)}")
    if solo_p: print(f"⚠️  Solo en PADRE ({len(solo_p)}): {sorted(solo_p)}")
    if not n_diff: print("\n✅ Ambas columnas usan el mismo catálogo.")

    # Gráfico comparativo de distribución
    fm = df[COL_M].value_counts(normalize=True).rename("Madre")
    fp = df[COL_P].value_counts(normalize=True).rename("Padre")
    comp = pd.concat([fm, fp], axis=1).fillna(0).sort_index() * 100

    fig, ax = plt.subplots(figsize=(13, max(5, len(comp) * 0.45)))
    comp.plot(kind="barh", ax=ax, color=["#4C72B0", "#DD8452"])
    ax.set_xlabel("Porcentaje (%)")
    ax.set_title("R06 — Educación Madre vs Padre (valores originales)", fontsize=11)
    plt.tight_layout()
    plt.savefig("r06_educacion_padres.png", dpi=120, bbox_inches="tight")
    plt.show()

    dq.add(VRule("R06", "Coherencia catálogos educación padres",
                 n_diff == 0, n_diff, len(cats_m | cats_p),
                 "Revisar fuente" if n_diff else "Sin acción",
                 f"{len(cats_m & cats_p)} cats comunes; {n_diff} divergentes"))
else:
    dq.add(VRule("R06", "Coherencia educación padres", True, 0, 0, "No disponible"))


### 🔹 R07 — Dominio válido: `FAMI_ESTRATOVIVIENDA`

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R07  DOMINIO VÁLIDO — FAMI_ESTRATOVIVIENDA
#      Colombia: Estrato 1-6 + Sin Estrato (zonas rurales)
#      Valores distintos → NaN para imputar en §6.
# ─────────────────────────────────────────────────────────────────────────────
VALID_EST = {"Estrato 1","Estrato 2","Estrato 3",
             "Estrato 4","Estrato 5","Estrato 6","Sin Estrato"}
COL = "FAMI_ESTRATOVIVIENDA"

if COL in df.columns:
    df[COL] = df[COL].astype(str).str.strip()
    mask = ~df[COL].isin(VALID_EST) & ~df[COL].isin({"nan","None","<NA>",""})
    n    = int(mask.sum())

    vc = df[COL].value_counts(dropna=False).reset_index()
    vc.columns = ["Valor","Conteo"]
    vc["En dominio"] = vc["Valor"].isin(VALID_EST).map({True:"✅", False:"❌"})
    display(vc)

    if n:
        df.loc[mask, COL] = np.nan
        print(f"\n⚠️  {n:,} registros fuera del dominio → NaN")
    else:
        print("\n✅ Todos los valores en el dominio.")

    dq.add(VRule("R07", f"Dominio {COL}", n == 0, n, len(df),
                 "→ NaN, imputar en §6" if n else "Sin acción",
                 "Dominio: Estrato 1-6 + Sin Estrato"))
else:
    dq.add(VRule("R07", f"Dominio {COL}", True, 0, 0, "No disponible"))


### 🔹 R08 — Variables numéricas del hogar: valores negativos o imposibles

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R08  VALORES NEGATIVOS EN VARIABLES NUMÉRICAS DEL HOGAR
#
#  Variables disponibles en df en este punto:
#    FAMI_CUARTOSHOGAR   : número de cuartos (entero ≥ 1)
#    FAMI_PERSONASHOGAR  : personas en el hogar (entero ≥ 1)
#
#  NOTA: INDICE_BIENES y EDAD no existen aún.
#        Los puntajes PUNT_* ya fueron validados en R05.
#
#  Un valor negativo o cero en estas variables es imposible físicamente.
#  Acción: → NaN para que §6 (imputación) los corrija con la mediana.
# ─────────────────────────────────────────────────────────────────────────────
HOGAR_COLS = {
    "FAMI_CUARTOSHOGAR"  : (1, 20),   # cuartos: mínimo 1, máximo razonable 20
    "FAMI_PERSONASHOGAR" : (1, 30),   # personas: mínimo 1, máximo razonable 30
}

total_bad = 0

print(f"{'Variable':<28}  {'Min obs':>8}  {'Max obs':>8}  {'Fuera de rango':>14}  Estado")
print("─" * 72)

for col, (lo, hi) in HOGAR_COLS.items():
    if col not in df.columns:
        print(f"   {col:<26}  (no disponible en df)")
        continue

    s   = pd.to_numeric(df[col], errors="coerce")
    bad = int(((s < lo) | (s > hi)).sum())
    total_bad += bad

    flag = "✅" if bad == 0 else "⚠️ "
    min_v = s.min() if not s.isna().all() else "—"
    max_v = s.max() if not s.isna().all() else "—"
    print(f"{flag} {col:<26}  {str(min_v):>8}  {str(max_v):>8}  {bad:>14,}  "
          + ("OK" if bad == 0 else f"→ NaN [{lo},{hi}]"))

    if bad:
        df.loc[(s < lo) | (s > hi), col] = np.nan

if total_bad == 0:
    print("\n✅ Todas las variables del hogar tienen valores en rango.")
else:
    print(f"\n⚠️  {total_bad:,} valores imposibles corregidos → NaN")

dq.add(VRule("R08", "Variables hogar sin valores imposibles",
             total_bad == 0, total_bad, len(df),
             f"→ NaN: {total_bad:,} registros fuera de rango" if total_bad else "Sin acción",
             "FAMI_CUARTOSHOGAR [1,20] | FAMI_PERSONASHOGAR [1,30]"))


### 🔹 R09 — Multicolinealidad categórica (Cramér's V > 0.80)

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R09  MULTICOLINEALIDAD CATEGÓRICA — CRAMÉR'S V > 0.80
#
#  Se evalúan SOLO las variables categóricas presentes en df en este punto.
#  INDICE_BIENES no existe aún (se crea en §7), por lo que no interfiere.
#  TARGET_COLS y COLUMNS_TO_DROP están definidas en §1 y son constantes.
#
#  Umbral 0.80: pares prácticamente redundantes.
#  Acción: reporte + recomendación de eliminación.
# ─────────────────────────────────────────────────────────────────────────────
from scipy.stats import chi2_contingency

def cramers_v(x, y):
    ct   = pd.crosstab(x.dropna(), y.dropna())
    if ct.shape[0] < 2 or ct.shape[1] < 2:
        return 0.0
    chi2 = chi2_contingency(ct, correction=False)[0]
    n    = ct.values.sum()
    d    = n * (min(ct.shape[0] - 1, ct.shape[1] - 1))
    return float(np.sqrt(chi2 / d)) if d > 0 else 0.0

THRESHOLD = 0.80

# Variables categóricas disponibles en df ahora (post-§4, pre-§6)
excluir = set(TARGET_COLS) | set(COLUMNS_TO_DROP)
cat_r09 = [
    c for c in df.columns
    if df[c].dtype.name in ("object", "category", "string")
    and c not in excluir
    and df[c].nunique(dropna=True) < 200
    and df[c].nunique(dropna=True) > 1
]

print(f"Variables categóricas evaluadas ({len(cat_r09)}): {cat_r09}")

if len(cat_r09) >= 2:
    SAMPLE = min(80_000, len(df))
    dfs = df[cat_r09].sample(SAMPLE, random_state=42)

    cv_mat = pd.DataFrame(index=cat_r09, columns=cat_r09, dtype=float)
    for c1 in cat_r09:
        for c2 in cat_r09:
            cv_mat.loc[c1, c2] = 1.0 if c1 == c2 else cramers_v(dfs[c1], dfs[c2])

    # Pares críticos
    critical = [
        {"Var1": c1, "Var2": c2, "Cramér V": round(cv_mat.loc[c1, c2], 3)}
        for i, c1 in enumerate(cat_r09)
        for c2 in cat_r09[i+1:]
        if cv_mat.loc[c1, c2] > THRESHOLD
    ]

    # Heatmap
    fig, ax = plt.subplots(figsize=(max(8, len(cat_r09)), max(6, len(cat_r09) - 1)))
    sns.heatmap(cv_mat.astype(float), ax=ax, cmap="YlOrRd",
                annot=True, fmt=".2f", vmin=0, vmax=1, linewidths=0.3,
                cbar_kws={"label": "Cramér's V", "shrink": 0.7})
    ax.set_title(f"R09 — Cramér's V entre variables categóricas originales\n"
                 f"(umbral crítico = {THRESHOLD})", fontsize=11)
    plt.tight_layout()
    plt.savefig("r09_cramers_v.png", dpi=120, bbox_inches="tight")
    plt.show()

    n_crit = len(critical)
    if n_crit:
        print(f"\n⚠️  {n_crit} par(es) con Cramér's V > {THRESHOLD}:")
        display(pd.DataFrame(critical).sort_values("Cramér V", ascending=False))
        print("   Acción recomendada: eliminar la variable con menor correlación con PUNT_GLOBAL")
    else:
        print(f"\n✅ Sin pares con Cramér's V > {THRESHOLD}")

    dq.add(VRule("R09", f"Sin multicolinealidad crítica (V > {THRESHOLD})",
                 n_crit == 0, n_crit,
                 len(cat_r09) * (len(cat_r09) - 1) // 2,
                 "Revisar y eliminar redundante" if n_crit else "Sin acción",
                 f"{len(cat_r09)} variables evaluadas"))
else:
    print("⚠️  No hay suficientes variables categóricas para evaluar.")
    dq.add(VRule("R09", "Multicolinealidad crítica", True, 0, 0, "No evaluable"))


### 🔹 R10 — Variables casi constantes (> 95 % un mismo valor)

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# R10  VARIABLES QUASI-CONSTANTES (> 95% un solo valor)
#
#  Una variable donde el 95%+ de los registros tienen el mismo valor
#  aporta información mínima al modelo.
#
#  Se evalúan TODAS las columnas disponibles en df en este punto,
#  excluyendo targets y columnas administrativas ya descartadas en §4.
#
#  NOTA: EDAD, ANIO, TRIMESTRE, INDICE_BIENES no existen aún → no se evalúan.
#  Acción: reporte. Si quasi-constante, considerar eliminar en §7.
# ─────────────────────────────────────────────────────────────────────────────
THRESH = 0.95

# Solo excluir lo que está definido y lo que ya no está en df
excluir_r10 = set(TARGET_COLS) | set(COLUMNS_TO_DROP)

# Columnas presentes en df que no son targets ni administrativas descartadas
input_cols = [c for c in df.columns if c not in excluir_r10]

resultados_r10 = []
quasi_const    = []

for col in input_cols:
    try:
        vc       = df[col].value_counts(normalize=True, dropna=False)
        if len(vc) == 0:
            continue
        top_val   = vc.index[0]
        top_ratio = float(vc.iloc[0])

        resultados_r10.append({
            "Variable"        : col,
            "Valor dominante" : str(top_val)[:40],
            "% dominante"     : round(top_ratio * 100, 2),
            "# Únicos"        : int(df[col].nunique()),
            "Estado"          : "⚠️  QUASI-CSTE" if top_ratio > THRESH else "✅ OK",
        })
        if top_ratio > THRESH:
            quasi_const.append(col)
    except Exception:
        continue

r10_df = (pd.DataFrame(resultados_r10)
          .sort_values("% dominante", ascending=False)
          .reset_index(drop=True))

print(f"Variables evaluadas: {len(r10_df)}")

if quasi_const:
    print(f"\n⚠️  Variables quasi-constantes ({len(quasi_const)}):")
    display(r10_df[r10_df["Estado"] == "⚠️  QUASI-CSTE"]
            [["Variable", "Valor dominante", "% dominante", "# Únicos"]])

# Visualización top 20
fig, ax = plt.subplots(figsize=(12, max(5, min(len(r10_df), 20) * 0.38)))
top20  = r10_df.head(20)
colors = ["#C44E52" if s == "⚠️  QUASI-CSTE" else "#4C72B0" for s in top20["Estado"]]
ax.barh(top20["Variable"], top20["% dominante"],
        color=colors, edgecolor="white", height=0.65)
ax.axvline(THRESH * 100, color="black", lw=1.5, ls="--",
           label=f"Umbral {THRESH*100:.0f}%")
ax.set_xlabel("% del valor dominante")
ax.set_title("R10 — Quasi-constancia de variables (top 20, valores originales)", fontsize=11)
ax.legend()
plt.tight_layout()
plt.savefig("r10_quasi_constantes.png", dpi=120, bbox_inches="tight")
plt.show()

if not quasi_const:
    print(f"\n✅ Ninguna variable supera el umbral del {THRESH*100:.0f}%.")

dq.add(VRule("R10", f"Sin variables quasi-constantes (>{THRESH*100:.0f}%)",
             len(quasi_const) == 0, len(quasi_const), len(input_cols),
             f"Revisar en §7: {quasi_const}" if quasi_const else "Sin acción",
             f"{len(input_cols)} variables evaluadas"))


### 📋 Reporte consolidado de validación

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 5.11  REPORTE CONSOLIDADO DE VALIDACIÓN
#       Aplicado sobre df POST-§4 (limpieza) y PRE-§6 (imputación).
#       Las correcciones → NaN se propagan automáticamente al pipeline.
# ─────────────────────────────────────────────────────────────────────────────
import matplotlib.patches as mpatches

dq.print_summary()
print()
display(dq.summary())

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

summary_df = dq.summary()
colors_bar = ["#55A868" if "PASS" in s else "#C44E52" for s in summary_df["Estado"]]

# Panel izq: estado por regla
labels = summary_df["ID"] + "  " + summary_df["Regla"].str[:30]
axes[0].barh(labels, [1] * len(summary_df),
             color=colors_bar, edgecolor="white")
axes[0].set_xlim(0, 1.35)
axes[0].set_title("Estado por regla de validación", fontsize=11, fontweight="bold")
axes[0].tick_params(axis="y", labelsize=8.5)
for i, (s, label) in enumerate(zip(summary_df["Estado"], labels)):
    axes[0].text(1.02, i, s, va="center", fontsize=9)
axes[0].set_xticks([])

green_p = mpatches.Patch(color="#55A868", label="✅ PASS")
red_p   = mpatches.Patch(color="#C44E52", label="⚠️  FAIL")
axes[0].legend(handles=[green_p, red_p], loc="lower right", fontsize=9)

# Panel der: % afectados (solo reglas con afectados > 0)
aff = summary_df[summary_df["% Afect."] > 0].copy()
if not aff.empty:
    axes[1].bar(aff["ID"], aff["% Afect."],
                color="#DD8452", edgecolor="white")
    for i, row in aff.iterrows():
        axes[1].text(aff.index.get_loc(i), row["% Afect."] + 0.01,
                     f"{row['% Afect.']:.2f}%", ha="center", fontsize=9)
    axes[1].set_xlabel("Regla")
    axes[1].set_ylabel("% registros afectados")
    axes[1].set_title("Impacto por regla (% sobre df)", fontsize=11, fontweight="bold")
else:
    axes[1].text(0.5, 0.5, "Sin registros afectados\n✅ Datos limpios",
                 ha="center", va="center", fontsize=14, transform=axes[1].transAxes,
                 color="#55A868")
    axes[1].set_title("Impacto por regla", fontsize=11, fontweight="bold")

fig.suptitle(
    "Dashboard de Calidad de Datos\n"
    "Evaluado POST-§4 (limpieza) | PRE-§6 (imputación) | Datos en unidades originales",
    fontsize=12, fontweight="bold", y=1.02
)
plt.tight_layout()
plt.savefig("dq_dashboard.png", dpi=120, bbox_inches="tight")
plt.show()

print(f"\n✅ Validación §5 completada. df listo para §6 (Imputación).")
print(f"   Shape actual de df: {df.shape}")
total_nan_new = df.isnull().sum().sum()
print(f"   NaN totales en df: {total_nan_new:,}  (serán imputados en §6)")


## 🩹 5. Imputación de Valores Faltantes

Estrategias aplicadas según el tipo de variable:

| Tipo de variable | Estrategia | Justificación |
|---|---|---|
| Numérica continua | **Mediana** | Robusta ante outliers |
| Categórica nominal | **Moda global** | Valor más frecuente |
| Educación madre/padre | **Moda por estrato** | Aprovecha correlación socioeconómica |


In [ ]:
# =============================================================================
# 5.1  INGENIERIA TEMPORAL - EDAD, ANIO Y TRIMESTRE
#
# ANIO: se calcula para extraer EDAD pero NO entra al modelo como variable
#       predictora. Razon: queremos predecir resultados de futuros estudiantes
#       que no conocen su anio de presentacion. Se imputa con la mediana del
#       dataset completo en la seccion de imputacion.
#
# TRIMESTRE: SI entra al modelo. Captura estacionalidad del examen
#            (primer semestre vs segundo semestre tienen patrones diferentes).
# =============================================================================

df["PERIODO"]   = df["PERIODO"].astype(str)
df["ANIO"]      = df["PERIODO"].str[:4].astype("Int16")
df["TRIMESTRE"] = df["PERIODO"].str[4:].astype("Int8")

# Fecha de referencia: 30 de junio del anio del examen
df["FECHA_REF"] = pd.to_datetime(
    df["ANIO"].astype("Int64").astype("string") + "-06-30", errors="coerce"
)

# Edad en anios
df["EDAD"] = (df["FECHA_REF"] - df["ESTU_FECHANACIMIENTO"]).dt.days / 365.25

# Filtrar edades imposibles para bachilleres
mask_bad = (df["EDAD"] < 12) | (df["EDAD"] > 60)
df.loc[mask_bad, "EDAD"] = np.nan
print(f"Edades imposibles -> NaN: {mask_bad.sum():,}")
print(df["EDAD"].describe().round(2).to_string())
print(f"\nTRIMESTRE distribucion:")
print(df["TRIMESTRE"].value_counts(dropna=False).sort_index().to_string())


In [ ]:
# =============================================================================
# 5.2  DIAGNÓSTICO DE NULOS ANTES DE IMPUTAR
# =============================================================================

vars_impute = [v for v in INDEPENDENT_VARS if v in df.columns]
null_before = df[vars_impute].isnull().sum()

print("── Nulos por variable independiente ────────────────────────────")
display(pd.DataFrame({
    "Nulos"  : null_before,
    "% Nulos": (df[vars_impute].isnull().mean() * 100).round(2),
    "Tipo"   : df[vars_impute].dtypes,
}).sort_values("Nulos", ascending=False)[lambda x: x["Nulos"] > 0])


In [ ]:
# =============================================================================
# 5.3  IMPUTACION NUMERICA (MEDIANA)
# REGLA: Nunca eliminar filas por una variable vacia.
# Se imputa SIEMPRE: mediana por grupo o mediana global como fallback.
# FAMI_PERSONASHOGAR: mediana por municipio para mayor precision.
# =============================================================================

num_vars = [
    v for v in vars_impute
    if pd.api.types.is_numeric_dtype(df[v]) and df[v].isnull().any()
]
if "EDAD" in df.columns and df["EDAD"].isnull().any() and "EDAD" not in num_vars:
    num_vars.append("EDAD")

for col in num_vars:
    n_null_antes = df[col].isnull().sum()

    if col == "FAMI_PERSONASHOGAR" and "COLE_MCPIO_UBICACION" in df.columns:
        # Imputacion por municipio para mayor precision demografica
        med_mcpio = df.groupby("COLE_MCPIO_UBICACION")[col].transform("median")
        df[col] = df[col].fillna(med_mcpio)
        # Fallback global si aun quedan nulos
        df[col] = df[col].fillna(df[col].median())
        print(f"  [{col:<30}] imputado por municipio + fallback global  (nulos: {n_null_antes} -> {df[col].isnull().sum()})")
    elif col == "FAMI_CUARTOSHOGAR" and "FAMI_ESTRATOVIVIENDA" in df.columns:
        # Imputacion por estrato
        med_estrato = df.groupby("FAMI_ESTRATOVIVIENDA")[col].transform("median")
        df[col] = df[col].fillna(med_estrato)
        df[col] = df[col].fillna(df[col].median())
        print(f"  [{col:<30}] imputado por estrato + fallback global  (nulos: {n_null_antes} -> {df[col].isnull().sum()})")
    else:
        med = df[col].median()
        df[col] = df[col].fillna(med)
        print(f"  [{col:<30}] mediana global = {med:.2f}  (nulos: {n_null_antes} -> {df[col].isnull().sum()})")

print("\nImputacion numerica completada.")
print(f"Filas conservadas: {len(df):,}  (sin eliminacion por nulos)")


In [ ]:
# =============================================================================
# 5.4  IMPUTACION CATEGORICA (MODA / MODA GRUPAL)
# REGLA: Nunca eliminar filas. Se imputa con moda grupal + fallback global.
# COLE_CARACTER: moda por area de ubicacion (urbano/rural).
# FAMI_EDUCACION*: moda por estrato de vivienda.
# =============================================================================

def impute_categorical(df, col, group_col=None):
    """
    Imputa categorica con moda grupal (si group_col existe)
    y moda global como fallback. Nunca elimina filas.
    """
    n_null = df[col].isnull().sum()
    if n_null == 0:
        return df

    if group_col and group_col in df.columns:
        group_mode = (
            df.groupby(group_col, observed=True)[col]
            .agg(lambda x: x.dropna().mode().iloc[0]
                 if not x.dropna().mode().empty else np.nan)
        )
        mask = df[col].isnull()
        df.loc[mask, col] = df.loc[mask, group_col].map(group_mode)

    # Fallback: moda global
    global_mode = df[col].dropna().mode()
    if not global_mode.empty:
        df[col] = df[col].fillna(global_mode.iloc[0])

    print(f"  [{col:<30}] nulos: {n_null} -> {df[col].isnull().sum()} | grupo={group_col or 'ninguno'}")
    return df


cat_vars = [
    v for v in vars_impute
    if df[v].dtype.name in ["category", "object", "string"] and df[v].isnull().any()
]

for col in cat_vars:
    if col == "COLE_CARACTER":
        group = "COLE_AREA_UBICACION"   # moda por area (urbano/rural)
    elif "EDUCACION" in col.upper():
        group = "FAMI_ESTRATOVIVIENDA"  # moda por estrato
    else:
        group = None
    df = impute_categorical(df, col, group_col=group)

print("\nImputacion categorica completada.")
print(f"Filas conservadas: {len(df):,}  (sin eliminacion por nulos)")

# Verificacion COLE_CARACTER
if "COLE_CARACTER" in df.columns:
    print("\n  COLE_CARACTER distribucion:")
    print(df["COLE_CARACTER"].value_counts(dropna=False).to_string())


In [ ]:
# =============================================================================
# 5.5  VERIFICACIÓN POST-IMPUTACIÓN
# =============================================================================
null_after = df[vars_impute].isnull().sum()

comparison = pd.DataFrame({
    "Antes"     : null_before,
    "Después"   : null_after,
    "Reducción" : null_before - null_after,
}).sort_values("Reducción", ascending=False)

display(comparison[comparison["Antes"] > 0])

remaining = null_after[null_after > 0]
print("\n✅ Sin nulos residuales en variables independientes." if remaining.empty
      else f"⚠️  Nulos residuales: {remaining.to_dict()}")


## 🔗 6. Análisis de Correlación y Multicolinealidad

| Métrica | Variables | Interpretación |
|---|---|---|
| **Pearson r** | Numéricas ↔ Numéricas | r ∈ [-1, 1]; |r| > 0.7 = alta correlación |
| **Cramér's V** | Categóricas ↔ Categóricas | V ∈ [0, 1]; V > 0.7 = posible redundancia |


In [ ]:
# =============================================================================
# 6.1  CORRELACIÓN DE PEARSON – VARIABLES NUMÉRICAS
# Se incluye PUNT_GLOBAL para identificar las variables más predictivas.
# =============================================================================

num_all = (
    df[vars_impute + ["PUNT_GLOBAL"]]
    .select_dtypes(include="number")
    .columns.tolist()
)
# Eliminar duplicados preservando orden
seen = set()
num_unique = [c for c in num_all if not (c in seen or seen.add(c))]

corr_matrix = df[num_unique].corr(method="pearson")

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    corr_matrix, annot=True, fmt=".2f", cmap="coolwarm",
    center=0, vmin=-1, vmax=1, linewidths=0.4, ax=ax,
    cbar_kws={"label": "Pearson r", "shrink": 0.8},
)
ax.set_title("Correlación de Pearson\n(Variables Numéricas + PUNT_GLOBAL)", fontsize=13, pad=15)
plt.tight_layout()
plt.savefig("pearson_correlation.png", dpi=130, bbox_inches="tight")
plt.show()

print("\n── Correlación de cada variable con PUNT_GLOBAL ───────────────")
print(
    corr_matrix["PUNT_GLOBAL"]
    .drop("PUNT_GLOBAL")
    .sort_values(key=abs, ascending=False)
    .round(4)
    .to_string()
)


In [ ]:
# =============================================================================
# 6.2  CRAMÉR'S V – VARIABLES CATEGÓRICAS
# Mide la asociación estadística entre pares de variables nominales/ordinales.
# Se trabaja con una muestra para eficiencia computacional.
# =============================================================================

def cramers_v(x: pd.Series, y: pd.Series) -> float:
    """
    Calcula Cramér's V entre dos Series categóricas.
    Retorna un valor entre 0 (sin asociación) y 1 (asociación perfecta).
    """
    ct   = pd.crosstab(x, y)
    chi2 = chi2_contingency(ct, correction=False)[0]
    n    = ct.values.sum()
    r, k = ct.shape
    denom = n * (min(k - 1, r - 1))
    return float(np.sqrt(chi2 / denom)) if denom > 0 else 0.0


cat_ind = [
    v for v in vars_impute
    if df[v].dtype.name == "category"
]

SAMPLE_SIZE = 150_000
df_sample = df[cat_ind].dropna().sample(min(SAMPLE_SIZE, len(df)), random_state=42)
print(f"Calculando Cramér's V en muestra de {len(df_sample):,} filas × {len(cat_ind)} variables...")

# Matriz de asociación
cv_df = pd.DataFrame(index=cat_ind, columns=cat_ind, dtype=float)
for c1 in cat_ind:
    for c2 in cat_ind:
        cv_df.loc[c1, c2] = 1.0 if c1 == c2 else cramers_v(df_sample[c1], df_sample[c2])

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(
    cv_df.astype(float), annot=True, fmt=".2f", cmap="Reds",
    vmin=0, vmax=1, linewidths=0.4, ax=ax,
    cbar_kws={"label": "Cramér's V", "shrink": 0.8},
)
ax.set_title("Multicolinealidad – Cramér's V\n(Variables Categóricas Independientes)", fontsize=13, pad=15)
plt.tight_layout()
plt.savefig("cramers_v_matrix.png", dpi=130, bbox_inches="tight")
plt.show()

# Reporte de pares problemáticos
print("\n🔎 Pares con Cramér's V > 0.70 (posible redundancia):")
pairs = [
    {"Variable 1": c1, "Variable 2": c2, "Cramér's V": round(cv_df.loc[c1, c2], 3)}
    for i, c1 in enumerate(cat_ind)
    for c2 in cat_ind[i+1:]
    if cv_df.loc[c1, c2] > 0.70
]
if pairs:
    display(pd.DataFrame(pairs).sort_values("Cramér's V", ascending=False))
else:
    print("  No se encontraron pares con V > 0.70 ✅")


## ⚙️ 7. Feature Engineering

In [ ]:
# =============================================================================
# 7.1  INDICE COMPUESTO DE BIENES DEL HOGAR (0-4)
# Suma binaria de los 4 bienes. Se verifica que la suma sea correcta.
# REGLA: si la columna no existe o tiene NaN, se cuenta como 0 (no como faltante).
# =============================================================================

available_assets = [c for c in ASSET_COLS if c in df.columns]
print(f"Columnas de bienes disponibles: {available_assets}")

for col in available_assets:
    # Normalizar a mayusculas sin espacios antes de mapear
    serie_norm = (
        df[col].astype("string").str.strip().str.upper()
        .replace({"<NA>": np.nan, "NONE": np.nan, "NAN": np.nan})
    )
    df[col + "_BIN"] = serie_norm.map({"SI": 1, "NO": 0}).fillna(0).astype("Int8")
    n_si  = (df[col + "_BIN"] == 1).sum()
    n_no  = (df[col + "_BIN"] == 0).sum()
    print(f"  {col:<30} SI={n_si:,}  NO={n_no:,}  nulos_tratados_como_0={serie_norm.isna().sum():,}")

bin_cols = [c + "_BIN" for c in available_assets]
df["INDICE_BIENES"] = df[bin_cols].sum(axis=1, min_count=0).astype(float)

print("\nINDICE_BIENES (0=ningun bien, 4=todos los bienes):")
print(df["INDICE_BIENES"].value_counts(dropna=False).sort_index().to_string())
print(f"  Media  : {df['INDICE_BIENES'].mean():.2f}")
print(f"  Nulos  : {df['INDICE_BIENES'].isnull().sum()}")


In [ ]:
# =============================================================================
# 7.2  WINSORIZACIÓN DE OUTLIERS EN PUNTAJES
# Se recortan valores extremos al percentil [0.5%, 99.5%] para evitar que
# errores de digitación afecten el entrenamiento de los modelos.
# =============================================================================

def winsorize(series: pd.Series, low: float = 0.005, high: float = 0.995) -> pd.Series:
    """Recorta los valores de una serie a los cuantiles [low, high]."""
    lo, hi = series.quantile(low), series.quantile(high)
    return series.clip(lower=lo, upper=hi)

for col in SCORE_COLS:
    if col in df.columns:
        df.loc[df[col] < 0, col] = np.nan      # puntajes negativos son imposibles
        df[col] = winsorize(df[col])

print("✅ Winsorización aplicada a puntajes (0.5%–99.5%):")
display(df[SCORE_COLS].describe().round(2))


In [ ]:
# =============================================================================
# 7.3  ELIMINACION DE VARIABLES REDUNDANTES
#
# ANIO: se elimina como variable predictora. Razon: los futuros estudiantes
#       no conocen su anio de presentacion. Solo se uso para calcular EDAD.
# TRIMESTRE: se conserva. Captura estacionalidad del examen.
# =============================================================================

COLS_TO_REMOVE = (
    ASSET_COLS                                      # reemplazadas por INDICE_BIENES
    + bin_cols                                       # columnas _BIN auxiliares
    + ["COLE_DEPTO_UBICACION",                       # ya se uso para filtrar Quindio
       "ESTU_PRIVADO_LIBERTAD",                      # valor casi constante
       "FECHA_REF", "ESTU_FECHANACIMIENTO",          # ya se extrajeron EDAD/TRIMESTRE
       "ANIO",                                       # no predictora para futuros estudiantes
       "PERIODO",]                                   # reemplazado por TRIMESTRE
)

removed_cols = [c for c in COLS_TO_REMOVE if c in df.columns]
df.drop(columns=removed_cols, inplace=True)

print(f"{len(removed_cols)} columnas eliminadas: {removed_cols}")
print(f"   Dimensiones actuales: {df.shape}")
print(f"   TRIMESTRE en df: {'TRIMESTRE' in df.columns}")
print(f"   ANIO en df     : {'ANIO' in df.columns}")


### 📍 7.4 — Variables contextuales agregadas (nivel municipio y colegio)

Se calculan **promedios de PUNT_GLOBAL** agregados por municipio y por colegio.  
Estas variables capturan el **efecto institucional y geografico** que va mas alla  
de las caracteristicas individuales del estudiante.

> Se calculan aqui (pre-encoding) porque necesitan `COLE_CODIGO` y `PUNT_GLOBAL`
> que son eliminados o no estan disponibles despues de §8.

In [ ]:
# =============================================================================
# 7.4  VARIABLES CONTEXTUALES AGREGADAS
# Se usa transform('mean') para preservar el indice y no perder filas.
# Deben calcularse ANTES del encoding porque necesitan COLE_CODIGO y PUNT_GLOBAL.
#
# PROM_GLOBAL_MCPIO  : promedio de PUNT_GLOBAL por municipio del colegio
# PROM_GLOBAL_COLEGIO: promedio de PUNT_GLOBAL por colegio (COLE_CODIGO)
#
# NOTA DE LEAKAGE: estos promedios incluyen al propio estudiante.
# Esto es aceptable para un modelo descriptivo/exploratorio. Si se requiere
# un pipeline de produccion estricto, calcular solo sobre el train set.
# =============================================================================

NUEVAS_CONTEXTUALES = []

# --- Promedio por municipio del colegio ---
if 'COLE_MCPIO_UBICACION' in df.columns and 'PUNT_GLOBAL' in df.columns:
    df['PROM_GLOBAL_MCPIO'] = (
        df.groupby('COLE_MCPIO_UBICACION')['PUNT_GLOBAL']
        .transform('mean')
    )
    NUEVAS_CONTEXTUALES.append('PROM_GLOBAL_MCPIO')
    print(f'  PROM_GLOBAL_MCPIO   : {df["PROM_GLOBAL_MCPIO"].notna().sum():,} valores '
          f'| media={df["PROM_GLOBAL_MCPIO"].mean():.2f} '
          f'| grupos={df["COLE_MCPIO_UBICACION"].nunique()}')
else:
    print('  AVISO: COLE_MCPIO_UBICACION o PUNT_GLOBAL no disponibles -> omitida')

# --- Promedio por colegio ---
if 'COLE_CODIGO' in df.columns and 'PUNT_GLOBAL' in df.columns:
    df['PROM_GLOBAL_COLEGIO'] = (
        df.groupby('COLE_CODIGO')['PUNT_GLOBAL']
        .transform('mean')
    )
    NUEVAS_CONTEXTUALES.append('PROM_GLOBAL_COLEGIO')
    print(f'  PROM_GLOBAL_COLEGIO : {df["PROM_GLOBAL_COLEGIO"].notna().sum():,} valores '
          f'| media={df["PROM_GLOBAL_COLEGIO"].mean():.2f} '
          f'| grupos={df["COLE_CODIGO"].nunique()}')
else:
    print('  AVISO: COLE_CODIGO o PUNT_GLOBAL no disponibles -> omitida')

# Registrar en INDEPENDENT_VARS para que pasen por encoding y scaling
for v in NUEVAS_CONTEXTUALES:
    if v not in INDEPENDENT_VARS:
        INDEPENDENT_VARS.append(v)

print(f'\nVariables contextuales creadas: {NUEVAS_CONTEXTUALES}')
print(f'INDEPENDENT_VARS actualizado: {len(INDEPENDENT_VARS)} variables')

# Distribucion rapida
if NUEVAS_CONTEXTUALES:
    import matplotlib.pyplot as plt
    fig, axes = plt.subplots(1, len(NUEVAS_CONTEXTUALES),
                             figsize=(7*len(NUEVAS_CONTEXTUALES), 4))
    if len(NUEVAS_CONTEXTUALES) == 1:
        axes = [axes]
    for ax, col in zip(axes, NUEVAS_CONTEXTUALES):
        df[col].dropna().plot.hist(ax=ax, bins=40, color='#4f8ef7', edgecolor='white')
        ax.set_title(col, fontsize=10, fontweight='bold')
        ax.set_xlabel('Promedio PUNT_GLOBAL', fontsize=9)
    plt.suptitle('7.4 Variables Contextuales - Distribucion',
                 fontsize=11, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


## 🏷️ 8. Codificación de Variables Categóricas

Los algoritmos de ML requieren entradas numéricas. Se aplican dos estrategias:

| Estrategia | Cuándo | Ejemplo |
|---|---|---|
| **OrdinalEncoder** | Variable con orden natural claro | Nivel educativo, estrato |
| **LabelEncoder** | Variable nominal (sin orden) | Género, calendario, jornada |


In [ ]:
# =============================================================================
# 8.1  PREPARACION DEL DATASET FINAL
# REGLA: Solo se elimina una fila si TODAS las variables de entrada son nulas.
# No se elimina por tener una sola variable vacia (ya fue imputada).
# =============================================================================

INDEPENDENT_VARS_FINAL = [v for v in INDEPENDENT_VARS if v in df.columns]
TARGETS_FINAL          = [t for t in TARGET_COLS       if t in df.columns]

df_model = df[INDEPENDENT_VARS_FINAL + TARGETS_FINAL].copy()

# Solo eliminar filas donde TODAS las variables X son nulas simultaneamente
mask_all_null = df_model[INDEPENDENT_VARS_FINAL].isnull().all(axis=1)
n_drop = mask_all_null.sum()
if n_drop > 0:
    df_model = df_model[~mask_all_null].copy()
    print(f"  Filas eliminadas (todas las X nulas): {n_drop:,}")
else:
    print("  Sin filas eliminadas (ninguna tiene todas las X nulas)")

print(f"Dataset de modelado: {df_model.shape[0]:,} filas x {df_model.shape[1]} cols")
print(f"   Variables X : {len(INDEPENDENT_VARS_FINAL)}")
print(f"   Variables y : {len(TARGETS_FINAL)}")
print(f"\n   Variables X incluidas:")
for v in INDEPENDENT_VARS_FINAL:
    n_null = df_model[v].isnull().sum()
    print(f"     {v:<35} nulos={n_null:,}")


In [ ]:
# =============================================================================
# 8.2  ORDINAL ENCODING - VARIABLES CON ORDEN NATURAL
# Las categorias deben estar SIN tildes (se normalizaron en 4.4).
# =============================================================================

# Categorias en orden ascendente sin tildes ni caracteres especiales
EDU_LEVELS = [
    "Ninguno",
    "Primaria incompleta",     "Primaria completa",
    "Secundaria (Bachillerato) incompleta", "Secundaria (Bachillerato) completa",
    "Tecnica o tecnologica incompleta",     "Tecnica o tecnologica completa",
    "Educacion profesional incompleta",     "Educacion profesional completa",
    "Postgrado",
]

ESTRATO_LEVELS = [
    "Sin Estrato",
    "Estrato 1", "Estrato 2", "Estrato 3",
    "Estrato 4", "Estrato 5", "Estrato 6",
]

ORDINAL_VARS = {
    "FAMI_EDUCACIONMADRE" : EDU_LEVELS,
    "FAMI_EDUCACIONPADRE" : EDU_LEVELS,
    "FAMI_ESTRATOVIVIENDA": ESTRATO_LEVELS,
}

encoders = {}

for col, cat_order in ORDINAL_VARS.items():
    if col not in df_model.columns:
        continue

    df_model[col] = df_model[col].astype(str).str.strip()

    # Verificar categorias presentes en datos
    present_raw = df_model[col].unique().tolist()
    print(f"  {col} - categorias en datos: {sorted([x for x in present_raw if x not in ['nan','None','<NA>']])}")

    present = [c for c in cat_order if c in df_model[col].unique()]
    extras  = [c for c in df_model[col].unique()
               if c not in cat_order and c not in ["nan", "None", "<NA>"]]
    all_cats = present + extras

    enc = OrdinalEncoder(
        categories=[all_cats],
        handle_unknown="use_encoded_value",
        unknown_value=-1,
        encoded_missing_value=-1,
    )
    df_model[col] = enc.fit_transform(df_model[[col]])
    encoders[col] = enc
    print(f"    -> OrdinalEncoding: {len(all_cats)} niveles -> rango [{df_model[col].min():.0f}, {df_model[col].max():.0f}]")

print("\nOrdinal Encoding completado.")


In [ ]:
# =============================================================================
# 8.3  LABEL ENCODING – VARIABLES NOMINALES
# LabelEncoder asigna un entero único a cada categoría sin implicar orden.
# Adecuado para variables nominales en modelos basados en árboles.
# =============================================================================

LABEL_VARS = [
    v for v in INDEPENDENT_VARS_FINAL
    if (df_model[v].dtype.name in ["category", "object", "string"])
    and v not in ORDINAL_VARS
]

for col in LABEL_VARS:
    if col not in df_model.columns:
        continue
    df_model[col] = df_model[col].astype(str)
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    encoders[col] = le
    print(f"  ✅ [{col:<30}] LabelEncoding  → {len(le.classes_)} clases")

print(f"\n✅ Label Encoding completado. Total encoders almacenados: {len(encoders)}")


### 🔧 8.5 — Interacciones economicas y transformaciones no lineales

Se crean variables derivadas a partir de las variables ya **codificadas numericamente**  
(post OrdinalEncoding de §8.2). Deben crearse antes del escalado para que  
MinMaxScaler las normalice en el mismo rango que el resto.

| Grupo | Variable | Formula | Captura |
|-------|----------|---------|--------|
| Economico | `ESTRATO_X_EDU_MADRE` | estrato x edu_madre | Capital social combinado |
| Economico | `ESTRATO_X_EDU_PADRE` | estrato x edu_padre | Capital social combinado |
| Hogar | `DENSIDAD_HOGAR` | personas / (cuartos + 1) | Hacinamiento |
| Economico | `INTERNET_X_EDU_MADRE` | internet x edu_madre | Acceso digital educado |
| Log | `LOG_PERSONAS` | log1p(personas) | Atenua asimetria |
| Log | `LOG_CUARTOS` | log1p(cuartos) | Atenua asimetria |

> Las transformaciones logaritmicas (`log1p`) se aplican porque personas y cuartos
> tienen distribuciones con cola derecha. `log1p` maneja el cero de forma segura.

In [ ]:
# =============================================================================
# 8.5  INTERACCIONES ECONOMICAS Y TRANSFORMACIONES NO LINEALES
#
# FAMI_ESTRATOVIVIENDA, FAMI_EDUCACIONMADRE, FAMI_EDUCACIONPADRE
# ya son numericas (OrdinalEncoder aplicado en 8.2).
# FAMI_TIENEINTERNET fue codificada como 0/1 en 8.3 (label encoding).
# FAMI_PERSONASHOGAR y FAMI_CUARTOSHOGAR son numericas desde la imputacion.
# =============================================================================

NUEVAS_FE = []
problemas = []

def safe_create(nombre, formula_fn, df_m):
    """Crea variable nueva manejando errores sin romper el pipeline."""
    try:
        serie = formula_fn(df_m)
        # Reemplazar inf por NaN
        serie = serie.replace([np.inf, -np.inf], np.nan)
        df_m[nombre] = serie
        n_null = serie.isna().sum()
        n_ok   = serie.notna().sum()
        print(f'  OK  {nombre:<30} '
              f'media={serie.mean():.3f}  '
              f'nulos={n_null:,} ({n_null/len(serie)*100:.1f}%)')
        return True
    except Exception as e:
        problemas.append((nombre, str(e)))
        print(f'  OMITIDA  {nombre:<24} -> {str(e)[:60]}')
        return False


print('--- Grupo 1: Interacciones economicas ---')

if safe_create(
    'ESTRATO_X_EDU_MADRE',
    lambda d: d['FAMI_ESTRATOVIVIENDA'] * d['FAMI_EDUCACIONMADRE'],
    df_model
):
    NUEVAS_FE.append('ESTRATO_X_EDU_MADRE')

if safe_create(
    'ESTRATO_X_EDU_PADRE',
    lambda d: d['FAMI_ESTRATOVIVIENDA'] * d['FAMI_EDUCACIONPADRE'],
    df_model
):
    NUEVAS_FE.append('ESTRATO_X_EDU_PADRE')

if safe_create(
    'DENSIDAD_HOGAR',
    lambda d: d['FAMI_PERSONASHOGAR'] / (d['FAMI_CUARTOSHOGAR'] + 1),
    df_model
):
    NUEVAS_FE.append('DENSIDAD_HOGAR')

# FAMI_TIENEINTERNET puede venir de label encoding (0/1) o no existir
internet_col = 'FAMI_TIENEINTERNET'
if internet_col in df_model.columns:
    if safe_create(
        'INTERNET_X_EDU_MADRE',
        lambda d: d[internet_col] * d['FAMI_EDUCACIONMADRE'],
        df_model
    ):
        NUEVAS_FE.append('INTERNET_X_EDU_MADRE')
else:
    print(f'  OMITIDA  INTERNET_X_EDU_MADRE -> {internet_col} no disponible')


print('\n--- Grupo 2: Transformaciones logaritmicas ---')

if safe_create(
    'LOG_PERSONAS',
    lambda d: np.log1p(d['FAMI_PERSONASHOGAR'].clip(lower=0)),
    df_model
):
    NUEVAS_FE.append('LOG_PERSONAS')

if safe_create(
    'LOG_CUARTOS',
    lambda d: np.log1p(d['FAMI_CUARTOSHOGAR'].clip(lower=0)),
    df_model
):
    NUEVAS_FE.append('LOG_CUARTOS')


# Registrar en INDEPENDENT_VARS_FINAL para que pasen por el escalado
for v in NUEVAS_FE:
    if v not in INDEPENDENT_VARS_FINAL:
        INDEPENDENT_VARS_FINAL.append(v)

print(f'\nVariables creadas exitosamente: {len(NUEVAS_FE)}')
print(f'  {NUEVAS_FE}')
if problemas:
    print(f'\nVariables omitidas por error: {len(problemas)}')
    for nm, err in problemas:
        print(f'  {nm}: {err}')
print(f'\nINDEPENDENT_VARS_FINAL: {len(INDEPENDENT_VARS_FINAL)} variables en total')

# Visualizacion rapida de distribucion de las nuevas variables
if NUEVAS_FE:
    cols_vis = [v for v in NUEVAS_FE if v in df_model.columns]
    n_cols   = len(cols_vis)
    n_rows   = (n_cols + 2) // 3
    fig, axes = plt.subplots(n_rows, 3,
                             figsize=(18, 4.5*n_rows),
                             constrained_layout=True)
    axes_flat = axes.flatten() if n_cols > 1 else [axes]

    grupos = {
        'ESTRATO_X_EDU_MADRE' : ('Interaccion economica', '#4f8ef7'),
        'ESTRATO_X_EDU_PADRE' : ('Interaccion economica', '#4f8ef7'),
        'DENSIDAD_HOGAR'      : ('Hacinamiento',          '#f0a840'),
        'INTERNET_X_EDU_MADRE': ('Interaccion economica', '#4f8ef7'),
        'LOG_PERSONAS'        : ('Log transform',         '#42d9a8'),
        'LOG_CUARTOS'         : ('Log transform',         '#42d9a8'),
    }

    for ax, col in zip(axes_flat, cols_vis):
        label, color = grupos.get(col, ('Variable', '#999'))
        df_model[col].dropna().plot.hist(
            ax=ax, bins=35, color=color, edgecolor='white', alpha=0.85
        )
        ax.set_title(f'{col}\n[{label}]', fontsize=9.5, fontweight='bold')
        ax.set_xlabel('Valor', fontsize=8.5)
        stats_txt = (f"media={df_model[col].mean():.3f}\n"
                     f"std ={df_model[col].std():.3f}\n"
                     f"nulos={df_model[col].isna().sum():,}")
        ax.text(0.97, 0.95, stats_txt, transform=ax.transAxes,
                ha='right', va='top', fontsize=7.5,
                bbox=dict(facecolor='white', alpha=0.8, pad=3, edgecolor='#ccc'))

    # Ocultar ejes sobrantes
    for ax in axes_flat[n_cols:]:
        ax.set_visible(False)

    fig.suptitle('8.5 Nuevas Variables: Interacciones + Log Transforms\n'
                 '(distribucion post-encoding, pre-escalado)',
                 fontsize=12, fontweight='bold')
    plt.savefig('fe_nuevas_variables.png', dpi=130, bbox_inches='tight')
    plt.show()
    print('fe_nuevas_variables.png guardado')


In [ ]:
# =============================================================================
# 8.4  VERIFICACIÓN POST-CODIFICACIÓN
# Todas las variables independientes deben ser de tipo numérico.
# =============================================================================

non_num = df_model[INDEPENDENT_VARS_FINAL].select_dtypes(exclude="number").columns.tolist()

if non_num:
    print(f"⚠️  Aún no numéricas: {non_num}")
else:
    print("✅ Todas las variables independientes son numéricas.")

display(df_model[INDEPENDENT_VARS_FINAL].dtypes.to_frame("Dtype tras codificación"))


## 📏 9. Escalado con MinMaxScaler

**MinMaxScaler** transforma cada variable al rango [0, 1]:

$$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

**¿Por qué escalar?** Algoritmos como KNN, SVM, Regresión Lineal con regularización
y Redes Neuronales son sensibles a la magnitud de las variables. Sin escalado,
una variable como `ANIO` (∼2020) dominaría sobre `EDAD` (∼17).

> Solo se escalan las variables independientes X. Las variables y (targets)
> no se escalan para preservar la interpretabilidad de las predicciones.


In [ ]:
# =============================================================================
# 9.1  SEPARAR X e y
# =============================================================================

X = df_model[INDEPENDENT_VARS_FINAL].copy()
y = df_model[TARGETS_FINAL].copy()

print("Rangos de X antes del escalado:")
display(X.agg(["min", "max"]).T.rename(columns={"min": "Mínimo", "max": "Máximo"}))


In [ ]:
# =============================================================================
# 9.2  APLICAR MINMAXSCALER
# Se guarda el scaler para poder invertir la transformación en producción
# (necesario para interpretar predicciones).
# =============================================================================

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled_arr = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled_arr, columns=INDEPENDENT_VARS_FINAL, index=X.index)

print("✅ MinMaxScaler aplicado.")
print("\nRango de X después del escalado (debe ser [0.000 – 1.000]):")
display(
    X_scaled.agg(["min", "max"]).T
    .rename(columns={"min": "Mínimo", "max": "Máximo"})
    .round(4)
)


In [ ]:
# =============================================================================
# 9.3  VISUALIZACIÓN ANTES vs DESPUÉS DEL ESCALADO
# =============================================================================

viz_cols = [c for c in ["EDAD", "FAMI_PERSONASHOGAR", "INDICE_BIENES", "ANIO"]
            if c in INDEPENDENT_VARS_FINAL]

fig, axes = plt.subplots(2, len(viz_cols), figsize=(5 * len(viz_cols), 8))

for i, col in enumerate(viz_cols):
    # Antes
    axes[0, i].hist(X[col].dropna(), bins=40, color="#4C72B0", edgecolor="white")
    axes[0, i].set_title(f"{col}\n(Original)", fontsize=10)
    axes[0, i].set_ylabel("Frecuencia" if i == 0 else "")

    # Después
    axes[1, i].hist(X_scaled[col].dropna(), bins=40, color="#DD8452", edgecolor="white")
    axes[1, i].set_title(f"{col}\nEscalado [0, 1]", fontsize=10)
    axes[1, i].set_xlabel("Valor")
    axes[1, i].set_ylabel("Frecuencia" if i == 0 else "")

fig.suptitle("Distribuciones antes y después de MinMaxScaler", fontsize=13, y=1.01)
plt.tight_layout()
plt.savefig("escalado_comparacion.png", dpi=130, bbox_inches="tight")
plt.show()


In [ ]:
# =============================================================================
# 9.4  DATASET FINAL UNIFICADO
# =============================================================================

df_final = pd.concat(
    [X_scaled.reset_index(drop=True), y.reset_index(drop=True)],
    axis=1
)

print(f"✅ Dataset final: {df_final.shape[0]:,} filas × {df_final.shape[1]} columnas")
print(f"   Variables X    : {len(INDEPENDENT_VARS_FINAL)}")
print(f"   Variables y    : {len(TARGETS_FINAL)}")
print(f"   Valores nulos  : {df_final.isnull().sum().sum()}")


## ✅ 10. Verificación Final y Exportación

In [ ]:
# =============================================================================
# 10.1  ESTADÍSTICAS FINALES
# =============================================================================

print("=" * 65)
print("  ESTADÍSTICAS DEL DATASET FINAL")
print("=" * 65)

print("\n── Variables independientes X (escaladas) ─────────────────────")
display(X_scaled.describe().round(4))

print("\n── Variables objetivo y (sin escalar) ─────────────────────────")
display(y.describe().round(2))


In [ ]:
# =============================================================================
# 10.2  EXPORTACIÓN
# Parquet: más eficiente para pipelines ML (preserva tipos, menor tamaño).
# CSV: compatibilidad universal.
# =============================================================================

try:
    df_final.to_parquet("icfes_procesado.parquet", index=False, compression="snappy")
    print("✅ Exportado: icfes_procesado.parquet")
except Exception as e:
    print(f"⚠️  Parquet no disponible: {e}")

df_final.to_csv("icfes_procesado.csv", index=False, encoding="utf-8")
print("✅ Exportado: icfes_procesado.csv")


In [ ]:
# =============================================================================
# 10.3  RESUMEN VISUAL DEL PIPELINE
# =============================================================================

steps = [
    ("1. Carga de datos",              f"{df_raw.shape[0]:,} filas  ×  {df_raw.shape[1]} columnas"),
    ("2. EDA",                         "Distribuciones, nulos, outliers"),
    ("3. Limpieza",                    "Tipos, normalización, encoding"),
    ("4. Imputación",                  "Mediana (num) | Moda grupal (cat)"),
    ("5. Análisis de correlación",     "Pearson + Cramér's V"),
    ("6. Feature Engineering",         "EDAD, ANIO, TRIMESTRE, INDICE_BIENES"),
    ("7. Eliminación de redundantes",  f"{len(removed_cols)} columnas removidas"),
    ("8. Codificación categórica",     f"Ordinal ({len(ORDINAL_VARS)}) + Label ({len(LABEL_VARS)})"),
    ("9. Escalado MinMaxScaler",       f"{len(INDEPENDENT_VARS_FINAL)} vars → [0, 1]"),
    ("10. Dataset final",              f"{df_final.shape[0]:,} filas  ×  {df_final.shape[1]} columnas"),
]

print("=" * 70)
print("  PIPELINE COMPLETO – RESUMEN")
print("=" * 70)
for step, detail in steps:
    print(f"  ✅  {step:<40} {detail}")
print("=" * 70)
print("\n🚀 Dataset listo para entrenar modelos de Machine Learning.")


---

## 🏁 12. Preparación Final del Dataset para Modelado

Esta sección realiza los últimos pasos antes de entrenar los modelos:

| Paso | Descripción |
|------|-------------|
| **12.1** | Limpieza estructural: eliminar columnas no escaladas y residuales |
| **12.2** | Codificación de la variable de salida `DESEMP_INGLES` (ordinal) |
| **12.3** | Auditoría final: tipos, nulos, rangos, duplicados |
| **12.4** | Serialización de objetos del pipeline (scaler, encoders) |
| **12.5** | Separación definitiva X / y por tipo de problema |
| **12.6** | Exportación del dataset y artefactos listos para modelado |


### 🧹 12.1 — Limpieza estructural: solo columnas escaladas y targets

In [ ]:
# =============================================================================
# 12.1  ELIMINAR COLUMNAS NO ESCALADAS Y RESIDUALES
#
# El dataset df_final puede contener columnas que no pasaron por el escalado
# (ej. variables categóricas sin codificar, columnas auxiliares de cálculo,
# o variables que quedaron del proceso de validación).
#
# Regla de oro para modelado:
#   ✅ Variables X → todas numéricas, en rango [0, 1] (MinMaxScaler)
#   ✅ Variables y → targets originales sin escalar (interpretables)
#   ❌ Cualquier otra columna → eliminar
# =============================================================================

# Variables que deben estar en el dataset final
EXPECTED_X = [c for c in INDEPENDENT_VARS_FINAL if c in df_final.columns]
EXPECTED_Y = [c for c in TARGETS_FINAL          if c in df_final.columns]
EXPECTED_ALL = EXPECTED_X + EXPECTED_Y

# Detectar columnas extra (no esperadas)
extra_cols = [c for c in df_final.columns if c not in EXPECTED_ALL]

print("── Diagnóstico de columnas ─────────────────────────────────────────────")
print(f"  Columnas esperadas (X): {len(EXPECTED_X)}")
print(f"  Columnas esperadas (y): {len(EXPECTED_Y)}")
print(f"  Columnas extra        : {len(extra_cols)}")

if extra_cols:
    print(f"\n  ⚠️  Columnas extra detectadas → serán eliminadas:")
    for c in extra_cols:
        print(f"     • {c}  (dtype: {df_final[c].dtype})")
    df_final.drop(columns=extra_cols, inplace=True)

# Reordenar: primero X, luego y
df_final = df_final[EXPECTED_X + EXPECTED_Y].copy()

print(f"\n✅ Dataset limpio:")
print(f"   Shape      : {df_final.shape}")
print(f"   Columnas X : {len(EXPECTED_X)}")
print(f"   Columnas y : {len(EXPECTED_Y)}")
print(f"\n   Columnas finales:")
for i, c in enumerate(df_final.columns):
    tag = "[X]" if c in EXPECTED_X else "[y]"
    print(f"   {i+1:>2}. {tag} {c}  ({df_final[c].dtype})")


### 🏷️ 12.2 — Codificación de la variable de salida `DESEMP_INGLES`

In [ ]:
# =============================================================================
# 12.2  CODIFICACIÓN DE DESEMP_INGLES (variable de salida categórica ordinal)
#
# DESEMP_INGLES es una variable de desempeño en inglés con niveles ordenados:
#   A- < A1 < A2 < B1 < B+ 
# (Marco Común Europeo de Referencia para las Lenguas - MCER)
#
# Se aplica OrdinalEncoder para preservar el orden jerárquico de los niveles.
# El encoder se guarda para poder invertir la transformación en predicción.
# =============================================================================

from sklearn.preprocessing import OrdinalEncoder
import numpy as np

COL_DESEMP = "DESEMP_INGLES"

if COL_DESEMP in df_final.columns:
    # Revisar los valores presentes antes de codificar
    print(f"Valores presentes en {COL_DESEMP}:")
    vc = df_final[COL_DESEMP].value_counts(dropna=False).reset_index()
    vc.columns = ["Nivel", "Conteo"]
    vc["% del total"] = (vc["Conteo"] / len(df_final) * 100).round(2)
    display(vc)

    # Orden oficial del MCER (ascendente)
    DESEMP_ORDER = ["A-", "A1", "A2", "B1", "B+"]

    # Categorías presentes + desconocidas al final
    present_cats = [c for c in DESEMP_ORDER if c in df_final[COL_DESEMP].dropna().unique()]
    extra_cats   = [c for c in df_final[COL_DESEMP].dropna().unique()
                    if c not in DESEMP_ORDER and str(c) not in ["nan", "None"]]
    all_cats     = present_cats + extra_cats

    enc_desemp = OrdinalEncoder(
        categories=[all_cats],
        handle_unknown="use_encoded_value",
        unknown_value=-1,
        encoded_missing_value=np.nan,
    )

    # Convertir a string para el encoder
    df_final[COL_DESEMP] = df_final[COL_DESEMP].astype(str).replace("nan", np.nan)
    df_final[[COL_DESEMP]] = enc_desemp.fit_transform(df_final[[COL_DESEMP]])

    # Guardar encoder en el diccionario de encoders del pipeline
    encoders[COL_DESEMP] = enc_desemp

    print(f"\n✅ {COL_DESEMP} codificada ordinalmente:")
    print(f"   Mapeo de niveles:")
    for i, cat in enumerate(enc_desemp.categories_[0]):
        print(f"     {cat:>4}  →  {i}")

    print(f"\n   Estadísticas tras codificación:")
    print(df_final[COL_DESEMP].describe().round(2).to_string())

    # Visualización de la distribución codificada
    import matplotlib.pyplot as plt
    import seaborn as sns

    fig, axes = plt.subplots(1, 2, figsize=(13, 4))

    # Distribución de frecuencias
    sns.countplot(
        x=df_final[COL_DESEMP].dropna(),
        ax=axes[0],
        palette="Blues_d",
        order=sorted(df_final[COL_DESEMP].dropna().unique()),
    )
    axes[0].set_title(f"Distribución de {COL_DESEMP} (codificada)", fontsize=11)
    axes[0].set_xlabel("Código numérico del nivel MCER")
    axes[0].set_ylabel("Frecuencia")

    # Etiquetas originales en el eje
    labels = [f"{i}
({cat})" for i, cat in enumerate(enc_desemp.categories_[0])]
    axes[0].set_xticklabels(labels, fontsize=9)

    # Boxplot de PUNT_INGLES por nivel de DESEMP_INGLES
    if "PUNT_INGLES" in df_final.columns:
        merged_plot = df_final[[COL_DESEMP, "PUNT_INGLES"]].dropna()
        sns.boxplot(
            x=COL_DESEMP,
            y="PUNT_INGLES",
            data=merged_plot,
            ax=axes[1],
            palette="Blues",
            order=sorted(merged_plot[COL_DESEMP].unique()),
        )
        axes[1].set_title(f"PUNT_INGLES por nivel de {COL_DESEMP}", fontsize=11)
        axes[1].set_xlabel("Nivel MCER (codificado)")
        axes[1].set_ylabel("Puntaje Inglés")
        xticklabels_1 = [f"{i}
({cat})" for i, cat in enumerate(enc_desemp.categories_[0])
                         if i in merged_plot[COL_DESEMP].unique()]
        axes[1].set_xticklabels(xticklabels_1, fontsize=9)
    else:
        axes[1].set_visible(False)

    plt.suptitle(f"Codificación Ordinal – {COL_DESEMP}", fontsize=13, y=1.02)
    plt.tight_layout()
    plt.savefig("desemp_ingles_codificado.png", dpi=130, bbox_inches="tight")
    plt.show()

else:
    print(f"⚠️  '{COL_DESEMP}' no encontrada en df_final.")


### 🔍 12.3 — Auditoría final del dataset

In [ ]:
# =============================================================================
# 12.3  AUDITORÍA FINAL — VERIFICACIÓN COMPLETA DEL DATASET
#
# Antes de entrenar cualquier modelo, el dataset debe cumplir:
#   1. Sin valores nulos en variables X
#   2. Todos los tipos de datos son numéricos
#   3. Variables X escaladas en [0, 1]
#   4. Sin filas duplicadas
#   5. Sin columnas constantes
#   6. Tamaño suficiente para entrenamiento
# =============================================================================

import pandas as pd
import numpy as np

print("=" * 70)
print("  AUDITORÍA FINAL DEL DATASET")
print("=" * 70)

EXPECTED_X_FINAL = [c for c in EXPECTED_X if c in df_final.columns]
EXPECTED_Y_FINAL = [c for c in EXPECTED_Y if c in df_final.columns]

audit_issues = []

# ── CHECK 1: Tipos de datos ──────────────────────────────────────────────────
non_numeric_X = df_final[EXPECTED_X_FINAL].select_dtypes(exclude="number").columns.tolist()
status_1 = "✅" if not non_numeric_X else "❌"
print(f"\n{status_1} CHECK 1 – Tipos numéricos en X")
if non_numeric_X:
    print(f"   Columnas no numéricas: {non_numeric_X}")
    audit_issues.append(f"Columnas no numéricas: {non_numeric_X}")
else:
    print(f"   Todas las {len(EXPECTED_X_FINAL)} variables X son numéricas.")

# ── CHECK 2: Nulos en X ──────────────────────────────────────────────────────
null_X = df_final[EXPECTED_X_FINAL].isnull().sum()
null_X_pos = null_X[null_X > 0]
status_2 = "✅" if null_X_pos.empty else "❌"
print(f"\n{status_2} CHECK 2 – Sin valores nulos en X")
if not null_X_pos.empty:
    print(f"   Columnas con nulos:")
    print(null_X_pos.to_string())
    audit_issues.append(f"Nulos en X: {null_X_pos.to_dict()}")
else:
    print(f"   0 nulos en las {len(EXPECTED_X_FINAL)} variables X.")

# ── CHECK 3: Rango [0, 1] en X ───────────────────────────────────────────────
X_min = df_final[EXPECTED_X_FINAL].min()
X_max = df_final[EXPECTED_X_FINAL].max()
out_of_range = ((X_min < -0.001) | (X_max > 1.001))
cols_oor = out_of_range[out_of_range].index.tolist()
status_3 = "✅" if not cols_oor else "⚠️"
print(f"\n{status_3} CHECK 3 – Rango [0, 1] en X (MinMaxScaler)")
if cols_oor:
    print(f"   Variables fuera de [0,1] (pueden ser NaN tratados):")
    for c in cols_oor:
        print(f"     {c}: min={X_min[c]:.4f}, max={X_max[c]:.4f}")
    audit_issues.append(f"Variables fuera de rango: {cols_oor}")
else:
    print(f"   Todas las variables X están en [0.0000, 1.0000].")

# ── CHECK 4: Duplicados ──────────────────────────────────────────────────────
n_dups = df_final.duplicated().sum()
status_4 = "✅" if n_dups == 0 else "⚠️"
print(f"\n{status_4} CHECK 4 – Sin filas duplicadas")
if n_dups > 0:
    print(f"   {n_dups:,} filas duplicadas detectadas ({n_dups/len(df_final)*100:.3f}%)")
    df_final.drop_duplicates(inplace=True)
    print(f"   → Eliminadas. Filas restantes: {len(df_final):,}")
    audit_issues.append(f"{n_dups} duplicados eliminados")
else:
    print(f"   Sin filas duplicadas en {len(df_final):,} registros.")

# ── CHECK 5: Variables constantes ────────────────────────────────────────────
var_zero = df_final[EXPECTED_X_FINAL].std()
const_cols = var_zero[var_zero == 0].index.tolist()
status_5 = "✅" if not const_cols else "❌"
print(f"\n{status_5} CHECK 5 – Sin variables constantes (std = 0)")
if const_cols:
    print(f"   Columnas constantes: {const_cols}")
    df_final.drop(columns=const_cols, inplace=True)
    EXPECTED_X_FINAL = [c for c in EXPECTED_X_FINAL if c not in const_cols]
    print(f"   → Eliminadas del dataset.")
    audit_issues.append(f"Columnas constantes eliminadas: {const_cols}")
else:
    print(f"   Ninguna variable tiene varianza cero.")

# ── CHECK 6: Tamaño del dataset ──────────────────────────────────────────────
MIN_ROWS = 1000
status_6 = "✅" if len(df_final) >= MIN_ROWS else "❌"
print(f"\n{status_6} CHECK 6 – Tamaño suficiente (mínimo {MIN_ROWS:,} registros)")
print(f"   Registros disponibles: {len(df_final):,}")
if len(df_final) < MIN_ROWS:
    audit_issues.append(f"Dataset muy pequeño: {len(df_final)} registros")

# ── CHECK 7: Nulos en y ───────────────────────────────────────────────────────
null_y = df_final[EXPECTED_Y_FINAL].isnull().sum()
null_y_pos = null_y[null_y > 0]
status_7 = "✅" if null_y_pos.empty else "⚠️"
print(f"\n{status_7} CHECK 7 – Nulos en y (variables objetivo)")
if not null_y_pos.empty:
    print(f"   Variables con nulos (normal para multioutput):")
    display(null_y_pos.to_frame("Nulos").assign(
        Porcentaje=lambda df: (df["Nulos"] / len(df_final) * 100).round(2)
    ))
else:
    print(f"   Sin nulos en las {len(EXPECTED_Y_FINAL)} variables objetivo.")

# ── Resumen final ─────────────────────────────────────────────────────────────
print("\n" + "=" * 70)
print(f"  RESULTADO DE AUDITORÍA: {'⚠️  ' + str(len(audit_issues)) + ' issue(s)' if audit_issues else '✅ DATASET APROBADO'}")
print("=" * 70)
if audit_issues:
    for i, issue in enumerate(audit_issues, 1):
        print(f"  {i}. {issue}")
else:
    print("  El dataset supera todos los checks de calidad.")
print(f"\n  📐 Shape final: {df_final.shape[0]:,} filas × {df_final.shape[1]} columnas")


### 💾 12.4 — Serialización del pipeline de preprocesamiento

In [ ]:
# =============================================================================
# 12.4  SERIALIZACIÓN DEL PIPELINE DE PREPROCESAMIENTO
#
# Se serializa con joblib (recomendado para objetos sklearn) y pickle como
# respaldo. Esto permite:
#   • Reutilizar las mismas transformaciones en producción sin re-entrenar
#   • Reproducir el preprocesamiento exacto sobre nuevos datos
#   • Invertir el escalado para interpretar predicciones (inverse_transform)
#
# Artefactos serializados:
#   pipeline_artefacts.joblib  ← objeto completo (recomendado)
#   scaler.joblib              ← MinMaxScaler individual
#   encoders.joblib            ← diccionario de encoders
# =============================================================================

import joblib
import pickle
import os
from datetime import datetime

# ── Construir el objeto completo del pipeline ─────────────────────────────────
pipeline_artefacts = {
    # Transformadores
    "scaler"             : scaler,        # MinMaxScaler fitted
    "encoders"           : encoders,      # dict {col: encoder} (OrdinalEncoder, LabelEncoder)

    # Metadata del pipeline
    "independent_vars"   : EXPECTED_X_FINAL,
    "target_vars"        : EXPECTED_Y_FINAL,
    "score_cols"         : SCORE_COLS,
    "asset_cols"         : ASSET_COLS,
    "yn_cols"            : YN_COLS,
    "na_values"          : NA_VALUES,
    "columns_to_drop"    : COLUMNS_TO_DROP,

    # Información del dataset
    "n_train_rows"       : len(df_final),
    "n_features"         : len(EXPECTED_X_FINAL),
    "n_targets"          : len(EXPECTED_Y_FINAL),
    "feature_range"      : (0, 1),
    "created_at"         : datetime.now().isoformat(),
    "dataset_version"    : "Resultados_únicos_Saber_11_20260224",
}

# ── Serializar con joblib ─────────────────────────────────────────────────────
OUTPUT_DIR = "."   # Cambiar a ruta de Google Drive si se desea persistencia

artefacts_path = os.path.join(OUTPUT_DIR, "pipeline_artefacts.joblib")
scaler_path    = os.path.join(OUTPUT_DIR, "scaler.joblib")
encoders_path  = os.path.join(OUTPUT_DIR, "encoders.joblib")

joblib.dump(pipeline_artefacts, artefacts_path, compress=3)
joblib.dump(scaler,             scaler_path,    compress=3)
joblib.dump(encoders,           encoders_path,  compress=3)

print("✅ Artefactos serializados con joblib:")
print(f"   📦 pipeline_artefacts.joblib  → {os.path.getsize(artefacts_path)/1024:.1f} KB")
print(f"   📦 scaler.joblib              → {os.path.getsize(scaler_path)/1024:.1f} KB")
print(f"   📦 encoders.joblib            → {os.path.getsize(encoders_path)/1024:.1f} KB")

# ── Verificar que los artefactos se pueden cargar correctamente ───────────────
loaded = joblib.load(artefacts_path)

assert loaded["n_features"]     == len(EXPECTED_X_FINAL), "Error: n_features no coincide"
assert loaded["n_train_rows"]   == len(df_final),          "Error: n_train_rows no coincide"
assert len(loaded["encoders"])  == len(encoders),          "Error: encoders no coinciden"

print("\n✅ Verificación de integridad: todos los artefactos cargados correctamente.")
print(f"\n── Contenido de pipeline_artefacts ───────────────────────────────────")
for k, v in loaded.items():
    if isinstance(v, (list, dict)):
        print(f"   {k:<25}: {type(v).__name__} con {len(v)} elementos")
    elif hasattr(v, '__class__'):
        print(f"   {k:<25}: {type(v).__name__}")
    else:
        print(f"   {k:<25}: {v}")


### ✂️ 12.5 — Separación definitiva X / y por tipo de problema

In [ ]:
# =============================================================================
# 12.5  SEPARACIÓN X / y POR TIPO DE PROBLEMA DE MODELADO
#
# Se generan distintas versiones de y según el tipo de modelo a entrenar:
#
#   y_regresion  → Puntajes numéricos continuos (PUNT_*)
#                  Modelos: Linear Regression, Random Forest Regressor, XGBoost
#
#   y_clasificacion → DESEMP_INGLES codificado ordinalmente (A- → B+)
#                     Modelos: Logistic Regression, Random Forest Classifier
#
#   y_multioutput → Todos los targets juntos para regresión multisalida
#                   Modelos: MultiOutputRegressor, Neural Network
# =============================================================================

import numpy as np

# ── X final (variables independientes escaladas) ──────────────────────────────
X_final = df_final[[c for c in EXPECTED_X_FINAL if c in df_final.columns]].copy()

# ── y – Regresión: puntajes numéricos ────────────────────────────────────────
score_targets = [c for c in SCORE_COLS if c in df_final.columns]
y_regresion   = df_final[score_targets].copy()

# ── y – Clasificación: desempeño en inglés ───────────────────────────────────
desemp_target = [c for c in ["DESEMP_INGLES"] if c in df_final.columns]
y_clasificacion = df_final[desemp_target].copy() if desemp_target else None

# ── y – Multioutput: todos los targets disponibles ───────────────────────────
y_multioutput = df_final[[c for c in EXPECTED_Y_FINAL if c in df_final.columns]].copy()

# ── Reporte de splits ─────────────────────────────────────────────────────────
print("=" * 65)
print("  SPLITS PARA MODELADO")
print("=" * 65)

print(f"\n📊 X_final         : {X_final.shape}  ← Variables independientes escaladas [0,1]")
print(f"\n📊 y_regresion     : {y_regresion.shape}")
for c in score_targets:
    stats = y_regresion[c].describe()
    print(f"   • {c:<35} media={stats['mean']:.1f}  std={stats['std']:.1f}")

if y_clasificacion is not None:
    print(f"\n📊 y_clasificacion : {y_clasificacion.shape}")
    print(f"   • DESEMP_INGLES  → {int(y_clasificacion['DESEMP_INGLES'].nunique())} clases")
    print(f"   Distribución de clases:")
    desemp_dist = y_clasificacion["DESEMP_INGLES"].value_counts(normalize=True).sort_index() * 100
    print(desemp_dist.round(2).to_string())

print(f"\n📊 y_multioutput   : {y_multioutput.shape}  ← Todos los targets")

# ── Verificar alineación de índices ───────────────────────────────────────────
assert len(X_final) == len(y_multioutput), "❌ X e y tienen diferente número de filas"
print(f"\n✅ Alineación X / y verificada: {len(X_final):,} registros en ambos.")


### 📤 12.6 — Exportación del dataset y artefactos listos para modelado

In [ ]:
# =============================================================================
# 12.6  EXPORTACIÓN FINAL DE TODOS LOS ARTEFACTOS
#
# Estructura de archivos generados:
# ─────────────────────────────────────────────────────────────────────────────
#  icfes_X_final.parquet          ← Variables independientes (escaladas)
#  icfes_y_regresion.parquet      ← Targets numéricos (PUNT_*)
#  icfes_y_clasificacion.parquet  ← Target categórico (DESEMP_INGLES)
#  icfes_y_multioutput.parquet    ← Todos los targets
#  icfes_dataset_completo.parquet ← Dataset completo X + y
#  pipeline_artefacts.joblib      ← Scaler + Encoders + Metadata
# =============================================================================

import joblib, os
import pandas as pd

exports = {
    "icfes_X_final.parquet"         : X_final,
    "icfes_y_regresion.parquet"     : y_regresion,
    "icfes_y_multioutput.parquet"   : y_multioutput,
    "icfes_dataset_completo.parquet": df_final,
}
if y_clasificacion is not None:
    exports["icfes_y_clasificacion.parquet"] = y_clasificacion

print("── Exportando archivos Parquet ─────────────────────────────────────────")
for fname, data in exports.items():
    try:
        data.to_parquet(fname, index=False, compression="snappy")
        size_kb = os.path.getsize(fname) / 1024
        print(f"   ✅ {fname:<45} {data.shape}  {size_kb:>7.1f} KB")
    except Exception as e:
        data.to_csv(fname.replace(".parquet", ".csv"), index=False)
        print(f"   ⚠️  {fname} (Parquet falló, exportado como CSV): {e}")

print("\n── Exportando CSVs de respaldo ─────────────────────────────────────────")
X_final.to_csv("icfes_X_final.csv",          index=False, encoding="utf-8")
y_regresion.to_csv("icfes_y_regresion.csv",   index=False, encoding="utf-8")
y_multioutput.to_csv("icfes_y_multioutput.csv", index=False, encoding="utf-8")
print("   ✅ CSVs exportados.")

print("\n── Artefactos del pipeline (ya serializados en 12.4) ───────────────────")
for f in ["pipeline_artefacts.joblib", "scaler.joblib", "encoders.joblib"]:
    if os.path.exists(f):
        print(f"   ✅ {f:<45} {os.path.getsize(f)/1024:>7.1f} KB")


In [ ]:
# =============================================================================
# 12.7  RESUMEN VISUAL — ESTADO FINAL DEL DATASET
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np

fig = plt.figure(figsize=(18, 12))
gs  = gridspec.GridSpec(2, 3, figure=fig, hspace=0.45, wspace=0.35)

# ── Panel 1: Distribución de X_final (boxplot escalado) ──────────────────────
ax1 = fig.add_subplot(gs[0, :2])
X_final.boxplot(ax=ax1, vert=False, patch_artist=True,
                boxprops=dict(facecolor="#4C72B0", alpha=0.6),
                medianprops=dict(color="#C44E52", lw=2))
ax1.set_title("Variables X — Distribución tras MinMaxScaler [0, 1]", fontsize=11, fontweight="bold")
ax1.set_xlabel("Valor escalado")
ax1.axvline(0, color="gray", lw=0.8, ls="--")
ax1.axvline(1, color="gray", lw=0.8, ls="--")

# ── Panel 2: Heatmap de correlación de X_final ───────────────────────────────
ax2 = fig.add_subplot(gs[0, 2])
corr_X = X_final.corr(method="pearson")
mask_upper = np.triu(np.ones_like(corr_X, dtype=bool), k=1)
sns.heatmap(corr_X, ax=ax2, cmap="coolwarm", center=0,
            vmin=-1, vmax=1, linewidths=0.3,
            cbar_kws={"shrink": 0.7, "label": "r"},
            xticklabels=True, yticklabels=True)
ax2.set_title("Correlación Pearson X_final", fontsize=11, fontweight="bold")
ax2.tick_params(axis="both", labelsize=7)

# ── Panel 3: Distribución de targets numéricos ───────────────────────────────
ax3 = fig.add_subplot(gs[1, :2])
y_plot = y_regresion[[c for c in score_targets if c in y_regresion.columns]]
y_plot.boxplot(ax=ax3, vert=False, patch_artist=True,
               boxprops=dict(facecolor="#55A868", alpha=0.6),
               medianprops=dict(color="#C44E52", lw=2))
ax3.set_title("Variables y (targets) — Puntajes sin escalar", fontsize=11, fontweight="bold")
ax3.set_xlabel("Puntaje")

# ── Panel 4: Nulos en y ───────────────────────────────────────────────────────
ax4 = fig.add_subplot(gs[1, 2])
null_y_pct = (y_multioutput.isnull().mean() * 100)
colors_null = ["#C44E52" if v > 0 else "#55A868" for v in null_y_pct]
ax4.barh(null_y_pct.index, null_y_pct.values, color=colors_null, edgecolor="white")
ax4.set_title("Nulos en variables y (%)", fontsize=11, fontweight="bold")
ax4.set_xlabel("% de nulos")
ax4.axvline(0, color="gray", lw=0.8)
for i, v in enumerate(null_y_pct.values):
    ax4.text(v + 0.1, i, f"{v:.1f}%", va="center", fontsize=8)

fig.suptitle("Estado Final del Dataset — Listo para Modelado", fontsize=14, y=1.01, fontweight="bold")
plt.savefig("estado_final_dataset.png", dpi=130, bbox_inches="tight")
plt.show()


In [ ]:
# =============================================================================
# 12.8  RESUMEN COMPLETO DEL PIPELINE — TODO EL PROYECTO
# =============================================================================

import os
from datetime import datetime

print("=" * 72)
print("  PIPELINE COMPLETO ICFES SABER 11 — RESUMEN FINAL")
print("=" * 72)

pipeline_summary = [
    # (Sección, Descripción, Resultado)
    ("§ 2",  "Carga de datos",              f"{df_raw.shape[0]:,} filas × {df_raw.shape[1]} cols"),
    ("§ 3",  "EDA",                         "Distribuciones, nulos, estadísticas"),
    ("§ 4",  "Limpieza",                    "Tipos → category, normalización strings"),
    ("§ 5",  "Imputación",                  "Mediana (num) | Moda grupal (cat)"),
    ("§ 6",  "Correlación",                 "Pearson + Cramér's V documentados"),
    ("§ 7",  "Feature Engineering",         "EDAD, ANIO, TRIMESTRE, INDICE_BIENES"),
    ("§ 8",  "Codificación",                f"OrdinalEncoder ({len(ORDINAL_VARS)}) + LabelEncoder ({len(LABEL_VARS)})"),
    ("§ 9",  "Escalado MinMaxScaler",        f"{len(EXPECTED_X_FINAL)} vars → [0, 1]"),
    ("§ 10", "Dataset procesado",           f"{df_final.shape[0]:,} filas"),
    ("§ 11", "Validación de calidad",       "10 reglas evaluadas"),
    ("§ 12.1","Limpieza estructural",        "Solo vars escaladas + targets"),
    ("§ 12.2","Codificación DESEMP_INGLES",  "OrdinalEncoder MCER (A- → B+)"),
    ("§ 12.3","Auditoría final",            "7 checks: tipos, nulos, rangos, dups"),
    ("§ 12.4","Serialización",              "pipeline_artefacts.joblib"),
    ("§ 12.5","Splits X/y",                f"X: {X_final.shape} | y_reg: {y_regresion.shape}"),
    ("§ 12.6","Exportación",               "Parquet + CSV + Joblib"),
]

for sec, desc, result in pipeline_summary:
    print(f"  ✅  {sec:<8} {desc:<32} → {result}")

print("=" * 72)
print(f"\n  📐 Dataset final listo para modelado:")
print(f"     X_final         : {X_final.shape[0]:>10,} filas × {X_final.shape[1]:>2} variables independientes")
print(f"     y_regresion     : {y_regresion.shape[0]:>10,} filas × {y_regresion.shape[1]:>2} targets continuos")
if y_clasificacion is not None:
    print(f"     y_clasificacion : {y_clasificacion.shape[0]:>10,} filas × {y_clasificacion.shape[1]:>2} targets categóricos")
print(f"     y_multioutput   : {y_multioutput.shape[0]:>10,} filas × {y_multioutput.shape[1]:>2} targets totales")

print(f"\n  🧠 Modelos recomendados para siguiente paso:")
print(f"     Regresión  → LinearRegression, RandomForestRegressor, XGBoostRegressor")
print(f"     Clasificac.→ LogisticRegression, RandomForestClassifier, XGBoostClassifier")
print(f"     Multioutput→ MultiOutputRegressor, MLPRegressor (Red Neuronal)")

print(f"\n  ⏱️  Pipeline generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 72)
print("  🚀 ¡Dataset 100% listo para entrenar modelos de Machine Learning!")
print("=" * 72)


---
## 🤖 13. Modelado — 6 Modelos × 6 Variables a Predecir

Para cada **variable objetivo** se entrenan **6 modelos de regresión** de forma independiente.
Esto genera **36 experimentos** que permiten identificar qué modelo es óptimo para cada puntaje.

| # | Modelo | Tipo |
|---|--------|------|
| M1 | **Ridge** | Lineal regularizado (baseline) |
| M2 | **KNN Regressor** | Basado en distancias |
| M3 | **Árbol de Decisión** | Árbol único, interpretable |
| M4 | **MLP** | Red neuronal multicapa |
| M5 | **Random Forest** | Ensemble de árboles (bagging) |
| M6 | **HistGradientBoosting** | Ensemble secuencial (boosting) |

**Variables objetivo (6):**
`PUNT_GLOBAL` · `PUNT_MATEMATICAS` · `PUNT_INGLES` · `PUNT_LECTURA_CRITICA` · `PUNT_C_NATURALES` · `PUNT_SOCIALES_CIUDADANAS`

**Métricas:** R² · RMSE · MAE · MAPE · Cross-Validation K=5


### ⚙️ 13.1 — Importaciones y configuración

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.1  LIBRERIAS DE MODELADO — 10 modelos
# ─────────────────────────────────────────────────────────────
import copy, time, warnings
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

from sklearn.linear_model   import Ridge
from sklearn.neighbors      import KNeighborsRegressor
from sklearn.tree           import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm            import SVR
from sklearn.ensemble       import (
    RandomForestRegressor,
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
)
from sklearn.model_selection import (
    train_test_split, KFold, cross_val_score,
    HalvingGridSearchCV
)
from sklearn.metrics import (
    r2_score, mean_squared_error,
    mean_absolute_error, mean_absolute_percentage_error
)
from sklearn.inspection import permutation_importance

import joblib, os

RANDOM_STATE = 42
N_FOLDS      = 5

print("Librerias cargadas.")


### 🗂️ 13.2 — Definición de los 6 modelos

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.2  CATALOGO DE 10 MODELOS
#
# M1_Ridge    Ridge (Baseline)
# M2_KNN      KNN Regressor
# M3_DTree    Arbol de Decision
# M4_MLP      MLP (128-64-32)
# M5_RF       Random Forest
# M6_HGB      HistGradientBoosting
# M7_GBR      GradientBoosting
# M8_SVR      SVR (omitido si dataset grande)
# M9_Deep     Red Neuronal Profunda (256-128-64-32)
# M10_Voting  Voting Regressor (RF + HGB + GBR)
# ─────────────────────────────────────────────────────────────

# SVR es lento con datasets grandes - omitir si hay mas de 20k filas
SVR_SKIP = len(X_final) > 20_000
if SVR_SKIP:
    print(f"  SVR omitido: dataset tiene {len(X_final):,} filas (> 20,000)")

# Voting base: RF + HGB + GBR (estimadores ligeros para que sea ejecutable)
_rf_base  = RandomForestRegressor(n_estimators=80, max_depth=10,
                                   min_samples_leaf=5, n_jobs=-1,
                                   random_state=RANDOM_STATE)
_hgb_base = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1,
                                            max_depth=6, random_state=RANDOM_STATE)
_gbr_base = GradientBoostingRegressor(n_estimators=80, learning_rate=0.1,
                                       max_depth=4, random_state=RANDOM_STATE)

MODELOS = [
    (
        "M1_Ridge",
        "Ridge (Baseline)",
        Ridge(alpha=1.0)
    ),
    (
        "M2_KNN",
        "KNN Regressor",
        KNeighborsRegressor(
            n_neighbors=7, weights="distance",
            metric="euclidean", n_jobs=-1
        )
    ),
    (
        "M3_DTree",
        "Arbol de Decision",
        DecisionTreeRegressor(
            max_depth=8, min_samples_leaf=20,
            random_state=RANDOM_STATE
        )
    ),
    (
        "M4_MLP",
        "MLP (128-64-32)",
        MLPRegressor(
            hidden_layer_sizes=(128, 64, 32),
            activation="relu", max_iter=500,
            early_stopping=True, validation_fraction=0.1,
            random_state=RANDOM_STATE
        )
    ),
    (
        "M5_RF",
        "Random Forest",
        RandomForestRegressor(
            n_estimators=150, max_depth=12,
            min_samples_leaf=5, n_jobs=-1,
            random_state=RANDOM_STATE
        )
    ),
    (
        "M6_HGB",
        "HistGradientBoosting",
        HistGradientBoostingRegressor(
            max_iter=200, learning_rate=0.05,
            max_depth=8, random_state=RANDOM_STATE
        )
    ),
    (
        "M7_GBR",
        "GradientBoosting",
        GradientBoostingRegressor(
            n_estimators=150, learning_rate=0.05,
            max_depth=5, subsample=0.8,
            random_state=RANDOM_STATE
        )
    ),
    (
        "M8_SVR",
        "SVR" if not SVR_SKIP else "SVR (omitido)",
        SVR(C=1.0, epsilon=0.1, kernel="rbf") if not SVR_SKIP else None
    ),
    (
        "M9_Deep",
        "Red Neuronal Profunda",
        MLPRegressor(
            hidden_layer_sizes=(256, 128, 64, 32),
            activation="relu", max_iter=600,
            early_stopping=True, validation_fraction=0.1,
            learning_rate_init=0.001,
            random_state=RANDOM_STATE
        )
    ),
    (
        "M10_Voting",
        "Voting Regressor",
        VotingRegressor(estimators=[
            ("rf",  _rf_base),
            ("hgb", _hgb_base),
            ("gbr", _gbr_base),
        ], n_jobs=-1)
    ),
]

TARGETS = [t for t in [
    "PUNT_GLOBAL", "PUNT_MATEMATICAS", "PUNT_INGLES",
    "PUNT_LECTURA_CRITICA", "PUNT_C_NATURALES", "PUNT_SOCIALES_CIUDADANAS"
] if t in y_regresion.columns]

n_efectivos = sum(1 for _,_,m in MODELOS if m is not None)
print(f"{n_efectivos} modelos efectivos x {len(TARGETS)} targets = {n_efectivos*len(TARGETS)} experimentos")
print()
print("  Modelos:")
for mid, mname, m in MODELOS:
    estado = "OMITIDO" if m is None else "OK"
    print(f"    {mid:<12}  {mname:<30}  [{estado}]")
print()
print("  Targets:")
for t in TARGETS:
    s = y_regresion[t].dropna()
    print(f"    {t:<35}  mu={s.mean():.1f}  sigma={s.std():.1f}  n={len(s):,}")


### ✂️ 13.3 — Partición Train / Test (80 / 20)

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.3  PARTICION UNICA 70/30
#
# Una sola particion 70/30, compartida por todos los modelos y targets.
# Garantiza comparaciones justas: mismo conjunto de test siempre.
# ─────────────────────────────────────────────────────────────

X_tr, X_te, y_tr_all, y_te_all = train_test_split(
    X_final, y_regresion,
    test_size=0.30, random_state=RANDOM_STATE, shuffle=True
)

print(f"  Train : {len(X_tr):>10,} filas  ({len(X_tr)/len(X_final)*100:.1f} %)")
print(f"  Test  : {len(X_te):>10,} filas  ({len(X_te)/len(X_final)*100:.1f} %)")
print(f"  X vars: {X_tr.shape[1]}")
print()

# Verificar distribución similar entre train y test
print(f"  {'Target':<35}  {'μ_train':>8}  {'μ_test':>8}  {'Δ':>6}  Estado")
print(f"  {'─'*35}  {'─'*8}  {'─'*8}  {'─'*6}  ──────")
for t in TARGETS:
    m_tr = y_tr_all[t].dropna().mean()
    m_te = y_te_all[t].dropna().mean()
    dif  = abs(m_tr - m_te)
    flag = "✅" if dif < 1.5 else "⚠️"
    print(f"  {flag} {t:<33}  {m_tr:>8.2f}  {m_te:>8.2f}  {dif:>6.2f}")


### 🔧 13.4 — Función de entrenamiento y evaluación

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.4  entrenar_modelo()
#
# Entrena un modelo para un target específico y retorna:
#   - R², RMSE, MAE, MAPE en test
#   - R² en Cross-Validation (K=5 folds)
#   - Tiempo de entrenamiento
#   - Modelo ya ajustado, predicciones y valores reales
# ─────────────────────────────────────────────────────────────

def entrenar_modelo(model_id, model_name, model,
                    target, X_train, X_test, y_train, y_test):
    # Modelo omitido (ej. SVR con dataset grande)
    if model is None:
        return None
    # Extraer serie y alinear índices (target puede tener NaN)
    y_tr = y_train[target].dropna()
    y_te = y_test[target].dropna()
    X_tr = X_train.loc[y_tr.index]
    X_te = X_test.loc[y_te.index]

    # Clonar para no contaminar experimentos posteriores
    m  = copy.deepcopy(model)
    t0 = time.time()
    m.fit(X_tr, y_tr)
    t_fit = round(time.time() - t0, 2)

    # Métricas en test
    y_pred = m.predict(X_te)
    y_real = y_te.values

    r2   = round(r2_score(y_real, y_pred), 4)
    rmse = round(np.sqrt(mean_squared_error(y_real, y_pred)), 4)
    mae  = round(mean_absolute_error(y_real, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_real, y_pred) * 100, 2)

    # Cross-Validation sobre train
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    cv = cross_val_score(
        copy.deepcopy(model), X_tr, y_tr,
        cv=kf, scoring="r2", n_jobs=-1
    )
    cv_mean = round(float(cv.mean()), 4)
    cv_std  = round(float(cv.std()),  4)

    return {
        "model_id"   : model_id,
        "model_name" : model_name,
        "target"     : target,
        "r2_test"    : r2,
        "rmse_test"  : rmse,
        "mae_test"   : mae,
        "mape_test"  : mape,
        "r2_cv_mean" : cv_mean,
        "r2_cv_std"  : cv_std,
        "r2_cv_fold" : list(cv),
        "tiempo_s"   : t_fit,
        # Para visualizaciones posteriores
        "modelo_fit" : m,
        "y_pred"     : y_pred,
        "y_real"     : y_real,
    }

print("✅ entrenar_modelo() lista.")


---
### 🔄 13.5 — Entrenamiento: un bloque por variable objetivo

Cada celda entrena los **6 modelos** para una variable objetivo específica
y muestra los resultados inmediatamente, permitiendo análisis progresivo.


In [ ]:
# ─────────────────────────────────────────────────────────────
# Almacenes globales
# ─────────────────────────────────────────────────────────────
RESULTADOS  = []                          # lista → DataFrame al final
MODELOS_FIT = {t: {} for t in TARGETS}   # {target: {model_id: resultado_completo}}

def entrenar_bloque(target):
    """Entrena los 6 modelos para `target` e imprime resultados."""
    y_tr_t = y_tr_all[[target]]
    y_te_t = y_te_all[[target]]

    n_tr = y_tr_t[target].dropna().__len__()
    n_te = y_te_t[target].dropna().__len__()

    print(f"\n{'═'*68}")
    print(f"  🎯  {target}  |  train={n_tr:,}  test={n_te:,}")
    print(f"{'═'*68}")
    print(f"  {'#':<3}  {'Modelo':<26}  {'R²_test':>8}  {'RMSE':>7}  "
          f"{'MAE':>7}  {'R²_CV (μ±σ)':>14}  {'t(s)':>5}")
    print(f"  {'─'*3}  {'─'*26}  {'─'*8}  {'─'*7}  {'─'*7}  {'─'*14}  {'─'*5}")

    for k, (mid, mname, model) in enumerate(MODELOS, 1):
        res = entrenar_modelo(
            mid, mname, model, target,
            X_tr, X_te, y_tr_t, y_te_t
        )
        MODELOS_FIT[target][mid] = res
        # Guardar versión sin arrays para el DataFrame
        row = {kk: vv for kk, vv in res.items()
               if kk not in ("modelo_fit", "y_pred", "y_real", "r2_cv_fold")}
        RESULTADOS.append(row)

        cv_str = f"{res['r2_cv_mean']:.4f}±{res['r2_cv_std']:.4f}"
        print(f"  {k:<3}  {mname:<26}  {res['r2_test']:>8.4f}  "
              f"{res['rmse_test']:>7.3f}  {res['mae_test']:>7.3f}  "
              f"{cv_str:>14}  {res['tiempo_s']:>5.1f}")

print("✅ entrenar_bloque() lista. Ejecuta las celdas siguientes.")


#### 📐 Bloque 1/6 — `PUNT_GLOBAL`

In [ ]:
if "PUNT_GLOBAL" in TARGETS:
    entrenar_bloque("PUNT_GLOBAL")

#### ➕ Bloque 2/6 — `PUNT_MATEMATICAS`

In [ ]:
if "PUNT_MATEMATICAS" in TARGETS:
    entrenar_bloque("PUNT_MATEMATICAS")

#### 🇬🇧 Bloque 3/6 — `PUNT_INGLES`

In [ ]:
if "PUNT_INGLES" in TARGETS:
    entrenar_bloque("PUNT_INGLES")

#### 📖 Bloque 4/6 — `PUNT_LECTURA_CRITICA`

In [ ]:
if "PUNT_LECTURA_CRITICA" in TARGETS:
    entrenar_bloque("PUNT_LECTURA_CRITICA")

#### 🔬 Bloque 5/6 — `PUNT_C_NATURALES`

In [ ]:
if "PUNT_C_NATURALES" in TARGETS:
    entrenar_bloque("PUNT_C_NATURALES")

#### 🏛️ Bloque 6/6 — `PUNT_SOCIALES_CIUDADANAS`

In [ ]:
if "PUNT_SOCIALES_CIUDADANAS" in TARGETS:
    entrenar_bloque("PUNT_SOCIALES_CIUDADANAS")

### 📊 13.6 — Tabla maestra de resultados

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.6  CONSTRUIR DATAFRAME MAESTRO Y MOSTRAR RANKING POR TARGET
# ─────────────────────────────────────────────────────────────

df_res = (
    pd.DataFrame(RESULTADOS)
    .sort_values(["target", "r2_test"], ascending=[True, False])
    .reset_index(drop=True)
)

print(f"DataFrame maestro: {df_res.shape[0]} filas × {df_res.shape[1]} cols\n")

RENAME = {
    "model_name": "Modelo",
    "r2_test"   : "R² Test",
    "r2_cv_mean": "R² CV μ",
    "r2_cv_std" : "R² CV σ",
    "rmse_test" : "RMSE",
    "mae_test"  : "MAE",
    "mape_test" : "MAPE %",
    "tiempo_s"  : "t (s)",
}

for target in TARGETS:
    sub = (df_res[df_res["target"] == target]
           .sort_values("r2_test", ascending=False)
           .reset_index(drop=True))
    sub.index += 1

    print(f"\n{'═'*60}")
    print(f"  🎯  {target}")
    print(f"{'═'*60}")
    display(
        sub[list(RENAME)].rename(columns=RENAME)
        .style
        .background_gradient(subset=["R² Test","R² CV μ"], cmap="YlGn",  vmin=0, vmax=1)
        .background_gradient(subset=["RMSE","MAE"],         cmap="YlOrRd_r")
        .format({
            "R² Test":"{:.4f}", "R² CV μ":"{:.4f}", "R² CV σ":"{:.4f}",
            "RMSE":"{:.3f}",    "MAE":"{:.3f}",     "MAPE %":"{:.2f}",
            "t (s)":"{:.1f}",
        })
        .highlight_max(subset=["R² Test"], color="#1e4d2b")
        .highlight_min(subset=["RMSE","MAE"], color="#1e4d2b")
        .set_properties(**{"font-size":"11px"})
    )


### 🗺️ 13.7 — Heatmaps: R², RMSE y MAE (modelo × target)

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.7  HEATMAPS COMPARATIVOS
#   Filas  = modelos  |  Columnas = targets
#   Borde verde = mejor celda de cada columna
# ─────────────────────────────────────────────────────────────

model_order = [m for _, m, _ in MODELOS]   # orden del catálogo

def pivot(col):
    p = df_res.pivot_table(
        index="model_name", columns="target",
        values=col, aggfunc="first"
    )
    return p.reindex(index=model_order, columns=TARGETS)

piv_r2   = pivot("r2_test")
piv_rmse = pivot("rmse_test")
piv_mae  = pivot("mae_test")

fig, axes = plt.subplots(1, 3, figsize=(24, 5))

specs = [
    (axes[0], piv_r2,   "YlGn",     "R² Test  ↑ mejor",    "max"),
    (axes[1], piv_rmse, "YlOrRd_r", "RMSE  ↓ menor mejor", "min"),
    (axes[2], piv_mae,  "PuRd_r",   "MAE   ↓ menor mejor", "min"),
]

for ax, piv, cmap, title, best in specs:
    sns.heatmap(
        piv, ax=ax, cmap=cmap,
        annot=True, fmt=".3f", linewidths=0.4,
        cbar_kws={"shrink": 0.75}, annot_kws={"size": 8.5}
    )
    ax.set_title(title, fontsize=11, fontweight="bold", pad=10)
    ax.set_xlabel("Variable objetivo", fontsize=9)
    ax.set_ylabel("Modelo", fontsize=9)
    ax.tick_params(axis="x", rotation=35, labelsize=8)
    ax.tick_params(axis="y", rotation=0,  labelsize=8)
    # Marcar mejor celda por columna con borde verde
    for j, col in enumerate(piv.columns):
        br = piv[col].idxmax() if best == "max" else piv[col].idxmin()
        ri = list(piv.index).index(br)
        ax.add_patch(plt.Rectangle(
            (j, ri), 1, 1, fill=False,
            edgecolor="#42d9a8", lw=2.5, zorder=5
        ))

fig.suptitle(
    "Comparación de métricas — 6 Modelos × 6 Targets  (▪ = mejor por columna)",
    fontsize=13, fontweight="bold", y=1.02
)
plt.tight_layout()
plt.savefig("heatmaps_metricas.png", dpi=130, bbox_inches="tight")
plt.show()


In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.8  BOXPLOTS DE CROSS-VALIDATION POR TARGET
#   Muestra la distribución de R² en los 5 folds.
#   IQR pequeño = modelo estable.  Mediana alta = modelo preciso.
# ─────────────────────────────────────────────────────────────

fig, axes = plt.subplots(2, 3, figsize=(20, 10))
axes = axes.flatten()
palette = sns.color_palette("Blues_d", len(MODELOS))

for i, target in enumerate(TARGETS):
    ax     = axes[i]
    data   = []
    labels = []

    for mid, mname, _ in MODELOS:
        folds = MODELOS_FIT[target][mid]["r2_cv_fold"]
        if folds:
            data.append(folds)
            labels.append(mname)

    if not data:
        ax.set_visible(False); continue

    bp = ax.boxplot(data, patch_artist=True, vert=True,
                    medianprops=dict(color="#e05c7a", lw=2.5),
                    whiskerprops=dict(color="#5c6280"),
                    capprops=dict(color="#5c6280"))
    for patch, col in zip(bp["boxes"], palette):
        patch.set_facecolor(col); patch.set_alpha(0.85)

    ax.set_xticks(range(1, len(labels)+1))
    ax.set_xticklabels(labels, fontsize=8, rotation=30, ha="right")
    ax.set_ylabel("R²  (fold)", fontsize=9)
    ax.set_title(target, fontsize=10, fontweight="bold")
    ax.axhline(0, color="#5c6280", lw=0.7, ls="--")
    ax.grid(axis="y", alpha=0.25)
    ax.tick_params(axis="y", labelsize=8)

fig.suptitle(
    f"Cross-Validation ({N_FOLDS} folds) — Distribución R² por Modelo y Target",
    fontsize=13, fontweight="bold", y=1.01
)
plt.tight_layout()
plt.savefig("cv_boxplots.png", dpi=130, bbox_inches="tight")
plt.show()


### 🏆 13.9 — Mejor modelo por variable objetivo

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.9  SELECCIÓN DEL MEJOR MODELO
#   Criterio 1: R² test máximo
#   Criterio 2 (desempate): RMSE mínimo
# ─────────────────────────────────────────────────────────────

MEJOR = {}

print(f"{'='*72}")
print(f"  MEJOR MODELO POR TARGET  (criterio: R² test máximo)")
print(f"{'='*72}")
print(f"  {'Target':<35}  {'Modelo':<24}  {'R²':>7}  {'RMSE':>7}  {'MAE':>7}  {'MAPE%':>6}")
print(f"  {'─'*35}  {'─'*24}  {'─'*7}  {'─'*7}  {'─'*7}  {'─'*6}")

for target in TARGETS:
    sub  = (df_res[df_res["target"] == target]
            .sort_values(["r2_test","rmse_test"], ascending=[False,True]))
    best = sub.iloc[0]
    mid  = best["model_id"]
    full = MODELOS_FIT[target][mid]

    MEJOR[target] = {
        "model_id"  : mid,
        "model_name": best["model_name"],
        "r2"        : best["r2_test"],
        "rmse"      : best["rmse_test"],
        "mae"       : best["mae_test"],
        "mape"      : best["mape_test"],
        "r2_cv"     : best["r2_cv_mean"],
        "r2_cv_std" : best["r2_cv_std"],
        "modelo_fit": full["modelo_fit"],
        "y_pred"    : full["y_pred"],
        "y_real"    : full["y_real"],
    }

    gap = sub["r2_test"].iloc[0] - sub["r2_test"].iloc[1]
    print(f"  {'🥇' if best['r2_test']==max(s['r2_test'] for s in sub.to_dict('records')) else '  '}"
          f" {target:<33}  {best['model_name']:<24}  "
          f"{best['r2_test']:>7.4f}  {best['rmse_test']:>7.3f}  "
          f"{best['mae_test']:>7.3f}  {best['mape_test']:>6.2f}"
          f"   Δ={gap:+.4f}")

print()
df_mejor = pd.DataFrame([{
    "Target":"", "Mejor Modelo":"", "R² Test":0.,
    "R² CV":0., "RMSE":0., "MAE":0., "MAPE %":0.
}])  # placeholder para display
df_mejor = pd.DataFrame([{
    "Target"      : t,
    "Mejor Modelo": v["model_name"],
    "R² Test"     : v["r2"],
    "R² CV"       : v["r2_cv"],
    "RMSE"        : v["rmse"],
    "MAE"         : v["mae"],
    "MAPE %"      : v["mape"],
} for t, v in MEJOR.items()])

display(
    df_mejor.style
    .background_gradient(subset=["R² Test","R² CV"], cmap="YlGn",  vmin=0, vmax=1)
    .background_gradient(subset=["RMSE","MAE"],       cmap="YlOrRd_r")
    .format({"R² Test":"{:.4f}","R² CV":"{:.4f}",
             "RMSE":"{:.3f}","MAE":"{:.3f}","MAPE %":"{:.2f}"})
)


### 📉 13.10 — Predicciones vs. Valores Reales

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.10  SCATTER PREDICCIONES vs REALES — MEJOR MODELO POR TARGET
#   Diagonal roja  = predicción perfecta
#   Línea verde    = tendencia real del modelo
# ─────────────────────────────────────────────────────────────

rng       = np.random.RandomState(RANDOM_STATE)
MAX_PTS   = 4_000
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
axes      = axes.flatten()

for i, target in enumerate(TARGETS):
    info   = MEJOR[target]
    y_real = info["y_real"]
    y_pred = info["y_pred"]
    ax     = axes[i]

    n   = min(MAX_PTS, len(y_real))
    idx = rng.choice(len(y_real), n, replace=False)

    ax.scatter(y_real[idx], y_pred[idx],
               alpha=0.18, s=9, color="#4f8ef7", rasterized=True)

    lo = min(y_real.min(), y_pred.min())
    hi = max(y_real.max(), y_pred.max())
    ax.plot([lo,hi],[lo,hi], "r--", lw=1.8, alpha=0.9, label="Ideal  y = ŷ")

    z  = np.polyfit(y_real, y_pred, 1)
    xs = np.linspace(lo, hi, 300)
    ax.plot(xs, np.poly1d(z)(xs), "#42d9a8", lw=1.5, alpha=0.85, label="Tendencia")

    ax.set_title(f"{target}\n{info['model_name']}", fontsize=9.5, fontweight="bold")
    ax.set_xlabel("Valor Real",  fontsize=8.5)
    ax.set_ylabel("Predicción",  fontsize=8.5)
    ax.tick_params(labelsize=7.5)
    ax.legend(fontsize=7.5, loc="upper left")
    ax.text(0.97, 0.05,
            f"R²  = {info['r2']:.3f}\nRMSE = {info['rmse']:.2f}\nMAE  = {info['mae']:.2f}",
            transform=ax.transAxes, ha="right", va="bottom", fontsize=8,
            bbox=dict(facecolor="#0d0f14", alpha=0.75, pad=4, edgecolor="#252a3a"),
            color="white")

fig.suptitle("Predicciones vs. Valores Reales — Mejor Modelo por Target",
             fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
plt.savefig("pred_vs_real.png", dpi=130, bbox_inches="tight")
plt.show()


In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.11  ANÁLISIS DE RESIDUOS   residuo = y_real − ŷ_pred
#
#   ✅ Residuos centrados en 0  → sin sesgo
#   ✅ Sin patrón en función de ŷ → homocedasticidad
#   Línea naranja = tendencia suavizada (debe ser ≈ 0 horizontal)
# ─────────────────────────────────────────────────────────────

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, target in enumerate(TARGETS):
    info   = MEJOR[target]
    y_real = info["y_real"]
    y_pred = info["y_pred"]
    res    = y_real - y_pred
    ax     = axes[i]

    n   = min(4_000, len(res))
    idx = rng.choice(len(res), n, replace=False)

    ax.scatter(y_pred[idx], res[idx],
               alpha=0.18, s=8, color="#dd8452", rasterized=True)
    ax.axhline(0,           color="#e05c7a", lw=1.8, ls="--", label="μ = 0")
    ax.axhline( res.std(),  color="#8fb4f8", lw=1,   ls=":",  alpha=0.7, label="+1σ")
    ax.axhline(-res.std(),  color="#8fb4f8", lw=1,   ls=":",  alpha=0.7, label="−1σ")

    # Tendencia suavizada
    ys  = np.argsort(y_pred)
    wnd = max(1, len(y_pred) // 60)
    smt = pd.Series((y_real - y_pred)[ys]).rolling(wnd, center=True, min_periods=1).mean()
    ax.plot(y_pred[ys], smt.values, "#f0a840", lw=1.5, alpha=0.8, label="Tendencia")

    ax.set_title(f"{target}\n({info['model_name']})", fontsize=9.5, fontweight="bold")
    ax.set_xlabel("Predicción",  fontsize=8.5)
    ax.set_ylabel("Residuo",     fontsize=8.5)
    ax.tick_params(labelsize=7.5)
    ax.legend(fontsize=7)
    ax.text(0.97, 0.97,
            f"μ = {res.mean():.2f}\nσ = {res.std():.2f}",
            transform=ax.transAxes, ha="right", va="top", fontsize=8,
            bbox=dict(facecolor="#0d0f14", alpha=0.75, pad=4, edgecolor="#252a3a"),
            color="white")

fig.suptitle("Análisis de Residuos — Mejor Modelo por Target",
             fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
plt.savefig("residuos.png", dpi=130, bbox_inches="tight")
plt.show()


### 🔬 13.12 — Feature Importance por target

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.12  IMPORTANCIA DE VARIABLES
#
#   Árboles / RF / HGB  → feature_importances_
#   Ridge               → |coef_|
#   KNN / MLP           → Permutation Importance
# ─────────────────────────────────────────────────────────────

def get_importancias(model, X_te, y_te):
    feats = list(X_te.columns)
    if hasattr(model, "feature_importances_"):
        imp = model.feature_importances_
    elif hasattr(model, "coef_"):
        imp = np.abs(model.coef_).flatten()[:len(feats)]
    else:
        pi  = permutation_importance(
            model, X_te, y_te,
            n_repeats=5, random_state=RANDOM_STATE, n_jobs=-1
        )
        imp = pi.importances_mean
    return pd.Series(np.abs(imp[:len(feats)]), index=feats).sort_values(ascending=False)


TOP_N = 12
fig, axes = plt.subplots(2, 3, figsize=(20, 13))
axes = axes.flatten()

for i, target in enumerate(TARGETS):
    info  = MEJOR[target]
    m     = info["modelo_fit"]
    ax    = axes[i]

    y_te_t = y_te_all[target].dropna()
    X_te_t = X_te.loc[y_te_t.index]

    try:
        imp = get_importancias(m, X_te_t, y_te_t.values)
        top = imp.head(TOP_N)

        cols = ["#4f8ef7"] + ["#1e3a5f"] * (len(top) - 1)
        ax.barh(top.index[::-1], top.values[::-1],
                color=cols[::-1], edgecolor="#1a1e2b", height=0.65)
        for bar, val in zip(ax.patches, top.values[::-1]):
            ax.text(val + top.max() * 0.01,
                    bar.get_y() + bar.get_height() / 2,
                    f"{val:.3f}", va="center", fontsize=7.5, color="#8fb4f8")
    except Exception as e:
        ax.text(0.5, 0.5, f"No disponible\n{e}",
                ha="center", va="center", fontsize=8, transform=ax.transAxes)

    ax.set_title(f"{target}\n({info['model_name']})", fontsize=9.5, fontweight="bold")
    ax.set_xlabel("Importancia", fontsize=8.5)
    ax.tick_params(axis="y", labelsize=8.5)
    ax.tick_params(axis="x", labelsize=7.5)

fig.suptitle(f"Top {TOP_N} Variables Predictoras — Mejor Modelo por Target",
             fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=130, bbox_inches="tight")
plt.show()


### 🏅 13.13 — Ranking global de modelos

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.13  RANKING GLOBAL
#   Panel izq : cuántas veces ganó cada modelo (victorias)
#   Panel der : R² promedio de cada modelo sobre los 6 targets
# ─────────────────────────────────────────────────────────────

model_order = [m for _, m, _ in MODELOS]
wins        = Counter(v["model_name"] for v in MEJOR.values())
win_vals    = [wins.get(n, 0) for n in model_order]

mean_r2_all = (
    df_res.groupby("model_name")["r2_test"]
    .mean().reindex(model_order).fillna(0)
)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Victorias
c1 = ["#42d9a8" if w > 0 else "#1e3a5f" for w in win_vals]
b1 = axes[0].barh(model_order, win_vals, color=c1,
                   edgecolor="#1a1e2b", height=0.55)
axes[0].set_xlabel("Targets donde fue el mejor modelo")
axes[0].set_title("¿Cuántas veces ganó cada modelo?", fontsize=11, fontweight="bold")
axes[0].set_xlim(0, max(win_vals) + 1.5)
axes[0].tick_params(axis="y", labelsize=9.5)
for bar, val in zip(b1, win_vals):
    if val > 0:
        axes[0].text(val + 0.05, bar.get_y() + bar.get_height()/2,
                     str(val), va="center", fontsize=12,
                     color="#42d9a8", fontweight="bold")

# R² promedio
best_r2 = mean_r2_all.max()
c2 = ["#4f8ef7" if v == best_r2 else "#1e3a5f" for v in mean_r2_all]
b2 = axes[1].barh(model_order, mean_r2_all.values, color=c2,
                   edgecolor="#1a1e2b", height=0.55)
axes[1].set_xlabel("R² promedio (6 targets)")
axes[1].set_title("R² Promedio por Modelo", fontsize=11, fontweight="bold")
axes[1].tick_params(axis="y", labelsize=9.5)
for bar, val in zip(b2, mean_r2_all.values):
    axes[1].text(val + 0.002, bar.get_y() + bar.get_height()/2,
                 f"{val:.3f}", va="center", fontsize=9,
                 color="#4f8ef7" if val == best_r2 else "#5c6280")

fig.suptitle("Ranking Global — 6 Modelos × 6 Targets",
             fontsize=13, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig("ranking_global.png", dpi=130, bbox_inches="tight")
plt.show()

print("\nResumen de victorias:")
for n, w, r2 in zip(model_order, win_vals, mean_r2_all.values):
    barra = "█" * w + "░" * (len(TARGETS) - w)
    print(f"  {n:<26}  {barra}  {w} victorias  |  R² prom = {r2:.4f}")


### 💾 13.14 — Serialización de los mejores modelos

### 🔧 13.13b — Ajuste suave de hiperparámetros (Top 3 modelos)

Se seleccionan los **3 mejores modelos** por R² promedio en todos los targets  
y se les aplica `HalvingGridSearchCV` — busqueda rapida que descarta candidatos  
malos en cada ronda, mucho mas eficiente que GridSearchCV completo.

- **factor=3**: descarta 2/3 de candidatos por ronda
- **CV=3 folds**: balanceo entre precision y velocidad
- **Grids reducidos**: solo 2-3 hiperparametros clave, max 27 combinaciones
- **Conservador**: solo reemplaza si R² optimizado >= R² base

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.13b  AJUSTE SUAVE DE HIPERPARAMETROS — TOP 3 MODELOS
# HalvingGridSearchCV: rapido y ejecutable en Colab sin timeout
# ─────────────────────────────────────────────────────────────

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
import copy, time

# ── Identificar top-3 modelos por R² promedio en todos los targets ───────────
if df_res is not None and len(df_res) > 0:
    ranking = (
        df_res.groupby("model_id")["r2"]
        .mean()
        .sort_values(ascending=False)
    )
    top3_ids = ranking.head(3).index.tolist()
    print(f"Top 3 modelos para ajuste: {top3_ids}")
    print(ranking.head(5).to_string())
else:
    top3_ids = ["M6_HGB", "M7_GBR", "M5_RF"]
    print(f"df_res no disponible, usando top3 por defecto: {top3_ids}")

# ── Grids reducidos por modelo ────────────────────────────────────────────────
# Solo 2-3 hiperparametros mas influyentes, max 27 combinaciones por modelo
GRIDS = {
    "M1_Ridge"  : {"alpha": [0.01, 0.1, 1.0, 10.0, 100.0]},
    "M2_KNN"    : {"n_neighbors": [5, 9, 15, 21],
                   "metric": ["euclidean", "manhattan"]},
    "M3_DTree"  : {"max_depth": [5, 8, 12, 16],
                   "min_samples_leaf": [10, 20, 40]},
    "M4_MLP"    : {"hidden_layer_sizes": [(128,64,32),(256,128,64)],
                   "alpha": [1e-4, 1e-3],
                   "max_iter": [500]},
    "M5_RF"     : {"n_estimators": [100, 200],
                   "max_depth": [8, 12, 16],
                   "min_samples_leaf": [3, 5]},
    "M6_HGB"    : {"max_iter": [100, 200, 300],
                   "learning_rate": [0.03, 0.05, 0.1],
                   "max_depth": [5, 8]},
    "M7_GBR"    : {"n_estimators": [100, 200],
                   "learning_rate": [0.03, 0.05, 0.1],
                   "max_depth": [3, 5]},
    "M8_SVR"    : {"C": [0.1, 1.0, 10.0], "epsilon": [0.05, 0.1]},
    "M9_Deep"   : {"hidden_layer_sizes": [(256,128,64,32),(512,256,128)],
                   "alpha": [1e-4, 1e-3]},
    "M10_Voting": {},   # VotingRegressor: no se ajusta directamente
}

# ── Funcion de ajuste para un modelo y un target ─────────────────────────────
def ajustar_modelo(mid, mname, modelo_base, target):
    """
    Ajusta hiperparametros con HalvingGridSearchCV.
    Retorna el mejor modelo solo si mejora el R² base.
    """
    if modelo_base is None or mid not in GRIDS or not GRIDS[mid]:
        return None

    grid = GRIDS[mid]
    y_tr = y_tr_all[target].dropna()
    X_tr_t = X_tr.loc[y_tr.index]

    t0 = time.time()
    try:
        search = HalvingGridSearchCV(
            copy.deepcopy(modelo_base),
            param_grid=grid,
            factor=3,
            cv=3,
            scoring="r2",
            n_jobs=-1,
            random_state=RANDOM_STATE,
            refit=True,
            verbose=0,
        )
        search.fit(X_tr_t, y_tr)

        # Evaluar en test
        y_te = y_te_all[target].dropna()
        X_te_t = X_te.loc[y_te.index]
        y_pred_opt = search.best_estimator_.predict(X_te_t)
        r2_opt = r2_score(y_te, y_pred_opt)

        elapsed = time.time() - t0
        return {
            "modelo_opt": search.best_estimator_,
            "best_params": search.best_params_,
            "r2_opt": r2_opt,
            "elapsed": elapsed,
        }
    except Exception as e:
        print(f"    Error en {mid}/{target}: {e}")
        return None

# ── Ejecutar ajuste para top-3 modelos en los 6 targets ──────────────────────
FITS_OPT = {}   # {target: {model_id: resultado}}
print()
print(f"{'Target':<35} {'Modelo':<15} {'R2_base':>8} {'R2_opt':>8} {'Delta':>7} {'Estado'}")
print("-" * 85)

for target in TARGETS:
    FITS_OPT[target] = {}
    y_te_t = y_te_all[target].dropna()
    X_te_t = X_te.loc[y_te_t.index]

    for mid in top3_ids:
        # Buscar instancia base en MODELOS
        modelo_base = next((m for i, n, m in MODELOS if i == mid), None)
        mname       = next((n for i, n, m in MODELOS if i == mid), mid)

        if modelo_base is None:
            continue

        # R2 base del entrenamiento original
        if target in FITS_BASE and mid in FITS_BASE[target]:
            info_base = FITS_BASE[target][mid]
            r2_base   = info_base.get("r2", 0.0)
            modelo_ajustado_base = info_base.get("modelo_fit")
        else:
            continue

        resultado = ajustar_modelo(mid, mname, modelo_ajustado_base, target)

        if resultado is None:
            continue

        r2_opt   = resultado["r2_opt"]
        delta    = r2_opt - r2_base
        usar_opt = r2_opt >= r2_base  # conservador: solo si mejora o iguala

        if usar_opt:
            FITS_OPT[target][mid] = resultado
            estado = "MEJOR" if delta > 0.001 else "IGUAL"
        else:
            estado = "BASE"

        print(f"{target:<35} {mid:<15} {r2_base:>8.4f} {r2_opt:>8.4f} {delta:>+7.4f}  {estado}")

print()
print("Ajuste de hiperparametros completado.")
print(f"  Modelos mejorados: {sum(len(v) for v in FITS_OPT.values())}")

# ── Actualizar MEJOR con modelos optimizados ──────────────────────────────────
n_actualizados = 0
for target, opts in FITS_OPT.items():
    if target not in MEJOR:
        continue
    mid_mejor = MEJOR[target]["model_id"]
    if mid_mejor in opts:
        opt = opts[mid_mejor]
        if opt["r2_opt"] >= MEJOR[target]["r2"]:
            MEJOR[target]["modelo_fit"] = opt["modelo_opt"]
            MEJOR[target]["r2"]         = opt["r2_opt"]
            MEJOR[target]["best_params"] = opt["best_params"]
            n_actualizados += 1
            print(f"  Actualizado MEJOR[{target}] con modelo optimizado  R2={opt['r2_opt']:.4f}")

print(f"\n{n_actualizados} modelos actualizados con version optimizada.")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.14  EXPORTACION DE LOS MEJORES MODELOS
# Nombre: icfes_mejor_{target}.joblib
# Incluye: modelo_fit, vars_sig, metricas, categorias de encoders
# ─────────────────────────────────────────────────────────────

import os
print("Exportando mejores modelos...")
OUTPUT_DIR = "."

for target, info in MEJOR.items():
    fname = os.path.join(OUTPUT_DIR, f"icfes_mejor_{target.lower()}.joblib")

    # Categorias texto de educacion madre/padre para la app (no niveles numericos)
    edu_categorias = {}
    for col in ["FAMI_EDUCACIONMADRE", "FAMI_EDUCACIONPADRE"]:
        if col in encoders:
            edu_categorias[col] = list(encoders[col].categories_[0])

    payload = {
        "target"          : target,
        "model_id"        : info["model_id"],
        "model_name"      : info["model_name"],
        "tipo"            : info.get("tipo", "ensemble"),
        "modelo_fit"      : info["modelo_fit"],   # objeto sklearn entrenado
        "vars_sig"        : info.get("vars_sig", list(X_tr.columns)),
        "n_vars"          : info.get("n_vars", X_tr.shape[1]),
        "metricas"        : {
            "r2_test"   : info["r2"],
            "rmse_test" : info["rmse"],
            "mae_test"  : info["mae"],
            "mape_test" : info["mape"],
            "r2_cv"     : info["r2_cv"],
            "r2_cv_std" : info["r2_cv_std"],
        },
        "y_pred_test"     : info.get("y_pred", []),
        "y_real_test"     : info.get("y_real", []),
        "n_train"         : len(X_tr),
        "n_test"          : len(X_te),
        "random_state"    : RANDOM_STATE,
        "features_all"    : list(X_tr.columns),
        "edu_categorias"  : edu_categorias,   # categorias texto para la app
        "exportado_en"    : pd.Timestamp.now().isoformat(),
    }

    joblib.dump(payload, fname, compress=3)
    kb = os.path.getsize(fname) / 1024

    # Verificacion critica: modelo_fit no debe ser None
    check = joblib.load(fname)
    estado = "OK" if check["modelo_fit"] is not None else "ERROR: modelo_fit es None"
    print(f"  {fname:<55} {kb:>7.1f} KB  [{estado}]")

# ── pipeline_artefacts.joblib actualizado ────────────────────────────────────
artefacts_path = os.path.join(OUTPUT_DIR, "pipeline_artefacts.joblib")
from datetime import datetime

pipeline_artefacts = {
    "scaler"          : scaler,
    "encoders"        : encoders,
    "independent_vars": EXPECTED_X_FINAL,
    "target_vars"     : EXPECTED_Y_FINAL,
    "score_cols"      : SCORE_COLS,
    "asset_cols"      : ASSET_COLS,
    "yn_cols"         : YN_COLS,
    "na_values"       : NA_VALUES,
    "edu_categorias"  : edu_categorias,   # categorias texto para la app
    "n_train_rows"    : len(X_tr),
    "n_features"      : len(EXPECTED_X_FINAL),
    "feature_range"   : (0, 1),
    "created_at"      : datetime.now().isoformat(),
}
joblib.dump(pipeline_artefacts, artefacts_path, compress=3)
kb = os.path.getsize(artefacts_path) / 1024
print(f"\n  pipeline_artefacts.joblib actualizado: {kb:.1f} KB")
print("\nExportacion completada.")


### 📋 13.15 — Resumen ejecutivo

In [ ]:
# ─────────────────────────────────────────────────────────────
# 13.15  RESUMEN EJECUTIVO FINAL
# ─────────────────────────────────────────────────────────────

from datetime import datetime

print("=" * 72)
print("  RESUMEN — 6 Modelos × 6 Targets  |  ICFES Saber 11")
print("=" * 72)
print(f"  Dataset  : {len(X_final):,} registros × {len(X_tr.columns)} variables X")
print(f"  Train    : {len(X_tr):,}  |  Test: {len(X_te):,}")
print(f"  CV       : {N_FOLDS}-Fold Stratified  |  scoring = R²")
print(f"  Modelos  : {len(MODELOS)}  |  Targets: {len(TARGETS)}")
print(f"  Experim. : {len(df_res)}")

print(f"\n  {'Target':<35}  {'Mejor Modelo':<24}  {'R²':>7}  {'RMSE':>7}  {'MAE':>7}")
print(f"  {'─'*35}  {'─'*24}  {'─'*7}  {'─'*7}  {'─'*7}")

all_r2 = [v["r2"] for v in MEJOR.values()]
for target in TARGETS:
    v  = MEJOR[target]
    mk = "🥇" if v["r2"] == max(all_r2) else "  "
    print(f"  {mk} {target:<33}  {v['model_name']:<24}  "
          f"{v['r2']:>7.4f}  {v['rmse']:>7.3f}  {v['mae']:>7.3f}")

wins   = Counter(v["model_name"] for v in MEJOR.values())
top_m  = wins.most_common(1)[0]
print(f"\n  R² medio (mejores)  : {np.mean(all_r2):.4f}")
print(f"  R² mínimo           : {min(all_r2):.4f}  "
      f"— {[t for t in TARGETS if MEJOR[t]['r2']==min(all_r2)][0]}")
print(f"  R² máximo           : {max(all_r2):.4f}  "
      f"— {[t for t in TARGETS if MEJOR[t]['r2']==max(all_r2)][0]}")
print(f"  Modelo dominante    : {top_m[0]}  ({top_m[1]}/{len(TARGETS)} targets)")
print(f"\n  {datetime.now().strftime('%Y-%m-%d  %H:%M:%S')}")
print("=" * 72)
